# Artificial Intelligence Nanodegree

## Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this IPython notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Use a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 6](#step6): Write your Algorithm
* [Step 7](#step7): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Dog Dataset

In the code cell below, we import a dataset of dog images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `dog_names` - list of string-valued dog breed names for translating labels

In [4]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


### Import Human Dataset

In the code cell below, we import a dataset of human images, where the file paths are stored in the numpy array `human_files`.

In [5]:
import random
random.seed(8675309)

# load filenames in shuffled human dataset
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)

# print statistics about the dataset
print('There are %d total human images.' % len(human_files))

There are 5750 total human images.


---
<a id='step1'></a>
## Step 1: Detect Humans

We use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.

In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

Number of faces detected: 1


In [7]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

def carregar_img_pasta(img):
    caminho_img = img.split('/')
    return (img + '/' + caminho_img[2] + '_0001.jpg')

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(carregar_img_pasta(human_files[3]))
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [8]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 

In [10]:
human_files_short = human_files[:100]
dog_files_short = train_files[:100]
# Do NOT modify the code above this line.

## TODO: Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.
human_detected = [h for h in human_files_short if face_detector(carregar_img_pasta(h))]
h_pc = len(human_detected) * 1.
print("Human percentage: {}%".format(h_pc))
dog_detected = [d for d in dog_files_short if face_detector(d)]
d_pc = len(dog_detected) * 1.
print("Dog percentage: {}%".format(d_pc))

Human percentage: 100.0%
Dog percentage: 11.0%


__Question 2:__ This algorithmic choice necessitates that we communicate to the user that we accept human images only when they provide a clear view of a face (otherwise, we risk having unneccessarily frustrated users!). In your opinion, is this a reasonable expectation to pose on the user? If not, can you think of a way to detect humans in images that does not necessitate an image with a clearly presented face?

__Answer:__

We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on each of the datasets.

In [29]:
## (Optional) TODO: Report the performance of another  
## face detection algorithm on the LFW dataset
### Feel free to use as many code cells as needed.
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

def load_resized(img_path):
    print("Img: {}".format(img_path))
    img = cv2.imread(img_path)
    return cv2.resize(img, (224, 224))

# humans_X = [load_resized(i) for i in human_files]
humans_X = []
for k in human_files[:800]:
    humans_X.append(load_resized(carregar_img_pasta(k)))
# a = (human_files[100]) )
print(humans_X)
dogs_X = [load_resized(i) for i in train_files]

Img: lfw/lfw/Tom_Watson/Tom_Watson_0001.jpg
Img: lfw/lfw/Rafael_Bielsa/Rafael_Bielsa_0001.jpg
Img: lfw/lfw/Jeanne_Anne_Schroeder/Jeanne_Anne_Schroeder_0001.jpg
Img: lfw/lfw/Jorge_Quiroga/Jorge_Quiroga_0001.jpg
Img: lfw/lfw/Scott_Gorelick/Scott_Gorelick_0001.jpg
Img: lfw/lfw/Chris_Reitsma/Chris_Reitsma_0001.jpg
Img: lfw/lfw/Eric_Christian_Olsen/Eric_Christian_Olsen_0001.jpg
Img: lfw/lfw/Julie_Andrews/Julie_Andrews_0001.jpg
Img: lfw/lfw/Luther_Htu/Luther_Htu_0001.jpg
Img: lfw/lfw/Alan_Jackson/Alan_Jackson_0001.jpg
Img: lfw/lfw/Renato_Soru/Renato_Soru_0001.jpg
Img: lfw/lfw/Christian_Fittipaldi/Christian_Fittipaldi_0001.jpg
Img: lfw/lfw/Charles_Richardson/Charles_Richardson_0001.jpg
Img: lfw/lfw/Katie_Boone/Katie_Boone_0001.jpg
Img: lfw/lfw/John_Thune/John_Thune_0001.jpg
Img: lfw/lfw/Maggie_Cheung/Maggie_Cheung_0001.jpg
Img: lfw/lfw/Kevin_Nealon/Kevin_Nealon_0001.jpg
Img: lfw/lfw/Rogelio_Ramos/Rogelio_Ramos_0001.jpg
Img: lfw/lfw/Aron_Ralston/Aron_Ralston_0001.jpg
Img: lfw/lfw/Ray_Nagin/Ray

Img: lfw/lfw/Julio_De_Brun/Julio_De_Brun_0001.jpg
Img: lfw/lfw/Kirk_Douglas/Kirk_Douglas_0001.jpg
Img: lfw/lfw/Bernard_Kerik/Bernard_Kerik_0001.jpg
Img: lfw/lfw/Yoriko_Kawaguchi/Yoriko_Kawaguchi_0001.jpg
Img: lfw/lfw/Rob_Moore/Rob_Moore_0001.jpg
Img: lfw/lfw/Kevin_Tarrant/Kevin_Tarrant_0001.jpg
Img: lfw/lfw/Yoon_Jin-Sik/Yoon_Jin-Sik_0001.jpg
Img: lfw/lfw/Vanessa_Redgrave/Vanessa_Redgrave_0001.jpg
Img: lfw/lfw/Roberto_Lavagna/Roberto_Lavagna_0001.jpg
Img: lfw/lfw/Tom_Miller/Tom_Miller_0001.jpg
Img: lfw/lfw/Bill_Readdy/Bill_Readdy_0001.jpg
Img: lfw/lfw/Amy_Brenneman/Amy_Brenneman_0001.jpg
Img: lfw/lfw/Farouk_Kaddoumi/Farouk_Kaddoumi_0001.jpg
Img: lfw/lfw/Ahmed_Lopez/Ahmed_Lopez_0001.jpg
Img: lfw/lfw/Michelangelo_Antonioni/Michelangelo_Antonioni_0001.jpg
Img: lfw/lfw/Guillaume_Cannet/Guillaume_Cannet_0001.jpg
Img: lfw/lfw/Queen_Noor/Queen_Noor_0001.jpg
Img: lfw/lfw/Stephan_Eberharter/Stephan_Eberharter_0001.jpg
Img: lfw/lfw/Charlotte_Church/Charlotte_Church_0001.jpg
Img: lfw/lfw/Mohammed_

Img: lfw/lfw/Jean_Nagel/Jean_Nagel_0001.jpg
Img: lfw/lfw/Juliette_Binoche/Juliette_Binoche_0001.jpg
Img: lfw/lfw/Kathy_Baker/Kathy_Baker_0001.jpg
Img: lfw/lfw/Nursultan_Nazarbayev/Nursultan_Nazarbayev_0001.jpg
Img: lfw/lfw/Hal_McCoy/Hal_McCoy_0001.jpg
Img: lfw/lfw/Dale_Earnhardt_Jr/Dale_Earnhardt_Jr_0001.jpg
Img: lfw/lfw/Gordon_Lightfoot/Gordon_Lightfoot_0001.jpg
Img: lfw/lfw/James_Murdoch/James_Murdoch_0001.jpg
Img: lfw/lfw/Jacques_Rogge/Jacques_Rogge_0001.jpg
Img: lfw/lfw/Hidetoshi_Nakata/Hidetoshi_Nakata_0001.jpg
Img: lfw/lfw/Daniel_Montenegro/Daniel_Montenegro_0001.jpg
Img: lfw/lfw/Neri_Marcore/Neri_Marcore_0001.jpg
Img: lfw/lfw/Tom_Coughlin/Tom_Coughlin_0001.jpg
Img: lfw/lfw/Daniela_Hantuchova/Daniela_Hantuchova_0001.jpg
Img: lfw/lfw/Pyar_Jung_Thapa/Pyar_Jung_Thapa_0001.jpg
Img: lfw/lfw/Bill_Herrion/Bill_Herrion_0001.jpg
Img: lfw/lfw/Karol_Kucera/Karol_Kucera_0001.jpg
Img: lfw/lfw/Maryn_McKenna/Maryn_McKenna_0001.jpg
Img: lfw/lfw/Richard_Hamilton/Richard_Hamilton_0001.jpg
Img: lfw

Img: lfw/lfw/Ben_Affleck/Ben_Affleck_0001.jpg
Img: lfw/lfw/Paul_McNulty/Paul_McNulty_0001.jpg
Img: lfw/lfw/Tex_Ritter/Tex_Ritter_0001.jpg
Img: lfw/lfw/Prospero_Pichay/Prospero_Pichay_0001.jpg
Img: lfw/lfw/Graham_Bentley/Graham_Bentley_0001.jpg
Img: lfw/lfw/Dennis_Johnson/Dennis_Johnson_0001.jpg
Img: lfw/lfw/Duane_Lee_Chapman/Duane_Lee_Chapman_0001.jpg
Img: lfw/lfw/Tom_Amstutz/Tom_Amstutz_0001.jpg
Img: lfw/lfw/Olympia_Dukakis/Olympia_Dukakis_0001.jpg
Img: lfw/lfw/David_Welch/David_Welch_0001.jpg
Img: lfw/lfw/Nicole_Parker/Nicole_Parker_0001.jpg
Img: lfw/lfw/Frank_Keating/Frank_Keating_0001.jpg
Img: lfw/lfw/Gennifer_Flowers/Gennifer_Flowers_0001.jpg
Img: lfw/lfw/Alberto_Ruiz_Gallardon/Alberto_Ruiz_Gallardon_0001.jpg
Img: lfw/lfw/Richard_Reid/Richard_Reid_0001.jpg
Img: lfw/lfw/Chris_Simon/Chris_Simon_0001.jpg
Img: lfw/lfw/Zhang_Yimou/Zhang_Yimou_0001.jpg
Img: lfw/lfw/Barbara_Felt-Miller/Barbara_Felt-Miller_0001.jpg
Img: lfw/lfw/Cliff_Ellis/Cliff_Ellis_0001.jpg
Img: lfw/lfw/Vince_Dooley/Vi

Img: lfw/lfw/Brent_Coles/Brent_Coles_0001.jpg
Img: lfw/lfw/Tyra_Banks/Tyra_Banks_0001.jpg
Img: lfw/lfw/Tim_Salmon/Tim_Salmon_0001.jpg
Img: lfw/lfw/Mark_Polansky/Mark_Polansky_0001.jpg
Img: lfw/lfw/Giulio_Andreotti/Giulio_Andreotti_0001.jpg
Img: lfw/lfw/Dominique_de_Villepin/Dominique_de_Villepin_0001.jpg
Img: lfw/lfw/Patrick_Kron/Patrick_Kron_0001.jpg
Img: lfw/lfw/William_Overlin/William_Overlin_0001.jpg
Img: lfw/lfw/Buford_Blount/Buford_Blount_0001.jpg
Img: lfw/lfw/Chen_Liang_Yu/Chen_Liang_Yu_0001.jpg
Img: lfw/lfw/David_Alpay/David_Alpay_0001.jpg
Img: lfw/lfw/Justin_Leonard/Justin_Leonard_0001.jpg
Img: lfw/lfw/Yasushi_Chimura/Yasushi_Chimura_0001.jpg
Img: lfw/lfw/Tina_Pisnik/Tina_Pisnik_0001.jpg
Img: lfw/lfw/Mathilda_Karel_Spak/Mathilda_Karel_Spak_0001.jpg
Img: lfw/lfw/Willie_Wilson/Willie_Wilson_0001.jpg
Img: lfw/lfw/Ray_Price/Ray_Price_0001.jpg
Img: lfw/lfw/Roy_Williams/Roy_Williams_0001.jpg
Img: lfw/lfw/Aileen_Riggin_Soule/Aileen_Riggin_Soule_0001.jpg
Img: lfw/lfw/Eve_Ensler/Eve_En

Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06108.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06516.jpg
Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00855.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05810.jpg
Img: dogImages/train/102.Manchester_terrier/Manchester_terrier_06806.jpg
Img: dogImages/train/119.Petit_basset_griffon_vendeen/Petit_basset_griffon_vendeen_07694.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04086.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01115.jpg
Img: dogImages/train/062.English_setter/English_setter_04407.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01822.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01279.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01245.jpg
Img: dogImages/train/121.Plott/Plott_07784.jpg
Img: dogImages/train/047.Chesapeake_bay_retriever/Chesapeake_bay_retriever_03365.jpg
Img: dogImages/train/076.Golden_retriever/Gold

Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04510.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01823.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04480.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03076.jpg
Img: dogImages/train/051.Chow_chow/Chow_chow_03667.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01928.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08061.jpg
Img: dogImages/train/036.Briard/Briard_02572.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07169.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03081.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07153.jpg
Img: dogImages/train/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05137.jpg
Img: dogImages/train/133.Yorkshire_terrier/Yorkshire_terrier_08318.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05047.jpg
Img: dogImages/train/036.Briard/Briard_0

Img: dogImages/train/009.American_water_spaniel/American_water_spaniel_00659.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02704.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04464.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03285.jpg
Img: dogImages/train/013.Australian_terrier/Australian_terrier_00916.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04859.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02683.jpg
Img: dogImages/train/122.Pointer/Pointer_07843.jpg
Img: dogImages/train/045.Cardigan_welsh_corgi/Cardigan_welsh_corgi_03188.jpg
Img: dogImages/train/067.Finnish_spitz/Finnish_spitz_04682.jpg
Img: dogImages/train/108.Norwegian_buhund/Norwegian_buhund_07107.jpg
Img: dogImages/train/092.Keeshond/Keeshond_06244.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03469.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04684.jpg
Im

Img: dogImages/train/066.Field_spaniel/Field_spaniel_04636.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06570.jpg
Img: dogImages/train/082.Havanese/Havanese_05582.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07571.jpg
Img: dogImages/train/030.Border_terrier/Border_terrier_02117.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05094.jpg
Img: dogImages/train/059.Doberman_pinscher/Doberman_pinscher_04159.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05117.jpg
Img: dogImages/train/062.English_setter/English_setter_04422.jpg
Img: dogImages/train/128.Smooth_fox_terrier/Smooth_fox_terrier_08126.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03290.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05829.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04726.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04363.jpg
Img: dogImages/train/071.German_shepherd_dog/Ger

Img: dogImages/train/111.Norwich_terrier/Norwich_terrier_07272.jpg
Img: dogImages/train/110.Norwegian_lundehund/Norwegian_lundehund_07191.jpg
Img: dogImages/train/081.Greyhound/Greyhound_05501.jpg
Img: dogImages/train/009.American_water_spaniel/American_water_spaniel_00647.jpg
Img: dogImages/train/071.German_shepherd_dog/German_shepherd_dog_04953.jpg
Img: dogImages/train/124.Poodle/Poodle_07942.jpg
Img: dogImages/train/022.Belgian_tervuren/Belgian_tervuren_01598.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02658.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04844.jpg
Img: dogImages/train/079.Great_pyrenees/Great_pyrenees_05389.jpg
Img: dogImages/train/028.Bluetick_coonhound/Bluetick_coonhound_01967.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02728.jpg
Img: dogImages/train/079.Great_pyrenees/Great_pyrenees_05382.jpg
Img: dogImages/train/088.Irish_water_spaniel/Irish_water_spaniel_05997.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_

Img: dogImages/train/057.Dalmatian/Dalmatian_04069.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04275.jpg
Img: dogImages/train/096.Labrador_retriever/Labrador_retriever_06452.jpg
Img: dogImages/train/007.American_foxhound/American_foxhound_00494.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04293.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05737.jpg
Img: dogImages/train/067.Finnish_spitz/Finnish_spitz_04677.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00687.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05896.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02676.jpg
Img: dogImages/train/115.Papillon/Papillon_07457.jpg
Img: dogImages/train/132.Xoloitzcuintli/Xoloitzcuintli_08291.jpg
Img: dogImages/train/007.American_foxhound/American_foxhound_00475.jpg
Img: dogImages/train/016.Beagle/Beagle_01181.jpg
Img: dogImages/train/106.Newfoundland/Newfoundland_06981.jpg
I

Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04485.jpg
Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01462.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00702.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07599.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04470.jpg
Img: dogImages/train/121.Plott/Plott_07787.jpg
Img: dogImages/train/062.English_setter/English_setter_04404.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06555.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08082.jpg
Img: dogImages/train/096.Labrador_retriever/Labrador_retriever_06488.jpg
Img: dogImages/train/004.Akita/Akita_00274.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07364.jpg
Img: dogImages/train/062.English_setter/English_setter_04373.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00666.jpg
Img: dogImages/train/102.Manchester_te

Img: dogImages/train/028.Bluetick_coonhound/Bluetick_coonhound_01955.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06610.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08054.jpg
Img: dogImages/train/050.Chinese_shar-pei/Chinese_shar-pei_03560.jpg
Img: dogImages/train/034.Boxer/Boxer_02365.jpg
Img: dogImages/train/085.Irish_red_and_white_setter/Irish_red_and_white_setter_05785.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01102.jpg
Img: dogImages/train/053.Cocker_spaniel/Cocker_spaniel_03787.jpg
Img: dogImages/train/104.Miniature_schnauzer/Miniature_schnauzer_06911.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04045.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06123.jpg
Img: dogImages/train/095.Kuvasz/Kuvasz_06419.jpg
Img: dogImages/train/001.Affenpinscher/Affenpinscher_00002.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05080.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01269.jpg
Img: dogImages/train/011.Austra

Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02962.jpg
Img: dogImages/train/036.Briard/Briard_02543.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01118.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02981.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03523.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02023.jpg
Img: dogImages/train/129.Tibetan_mastiff/Tibetan_mastiff_08177.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03279.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02266.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05823.jpg
Img: dogImages/train/133.Yorkshire_terrier/Yorkshire_terrier_08341.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06541.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01120.jpg
Img: dogImages/train/106.Newfoundland/Newfoundland_06974.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06839.jpg
Img: dogImages/train/084.Icelandi

Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02708.jpg
Img: dogImages/train/028.Bluetick_coonhound/Bluetick_coonhound_01961.jpg
Img: dogImages/train/034.Boxer/Boxer_02418.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08068.jpg
Img: dogImages/train/067.Finnish_spitz/Finnish_spitz_04664.jpg
Img: dogImages/train/110.Norwegian_lundehund/Norwegian_lundehund_07213.jpg
Img: dogImages/train/118.Pembroke_welsh_corgi/Pembroke_welsh_corgi_07641.jpg
Img: dogImages/train/119.Petit_basset_griffon_vendeen/Petit_basset_griffon_vendeen_07687.jpg
Img: dogImages/train/058.Dandie_dinmont_terrier/Dandie_dinmont_terrier_04112.jpg
Img: dogImages/train/106.Newfoundland/Newfoundland_07034.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02187.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04759.jpg
Img: dogImages/train/006.American_eskimo_dog/American_eskimo_dog_00408.jpg
Img: dogImages/train/036.Briard/Briard_02552.jpg
Img: dogImages/train/033.Bouvier_des_flandres/

Img: dogImages/train/015.Basset_hound/Basset_hound_01065.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01226.jpg
Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04501.jpg
Img: dogImages/train/101.Maltese/Maltese_06723.jpg
Img: dogImages/train/054.Collie/Collie_03816.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02192.jpg
Img: dogImages/train/078.Great_dane/Great_dane_05316.jpg
Img: dogImages/train/105.Neapolitan_mastiff/Neapolitan_mastiff_06965.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04266.jpg
Img: dogImages/train/036.Briard/Briard_02586.jpg
Img: dogImages/train/101.Maltese/Maltese_06767.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05067.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06824.jpg
Img: dogImages/train/037.Brittany/Brittany_02626.jpg
Img: dogImages/train/003.Airedale_terrier/Airedale_terrier_00199.jpg
Img: dogImages/train/054.Collie/Collie_03841.jpg
Img: dogImages/train/114.Otterhound/Otte

Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05732.jpg
Img: dogImages/train/066.Field_spaniel/Field_spaniel_04615.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01297.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_03034.jpg
Img: dogImages/train/016.Beagle/Beagle_01125.jpg
Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01455.jpg
Img: dogImages/train/044.Cane_corso/Cane_corso_03169.jpg
Img: dogImages/train/014.Basenji/Basenji_01000.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02685.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05724.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02720.jpg
Img: dogImages/train/051.Chow_chow/Chow_chow_03623.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01892.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07556.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_03027.jpg
Img: dogImages/train/082.Havanese/Havanese_05621.jpg
Img: dogImages/train/033.Bouvier_d

Img: dogImages/train/014.Basenji/Basenji_00986.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04549.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01909.jpg
Img: dogImages/train/099.Lhasa_apso/Lhasa_apso_06657.jpg
Img: dogImages/train/022.Belgian_tervuren/Belgian_tervuren_01594.jpg
Img: dogImages/train/102.Manchester_terrier/Manchester_terrier_06774.jpg
Img: dogImages/train/036.Briard/Briard_02516.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07293.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06124.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03058.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00693.jpg
Img: dogImages/train/101.Maltese/Maltese_06739.jpg
Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04522.jpg
Img: dogImages/train/106.Newfoundland/Newfoundland_07017.jpg
Img: dogImages/train/078.Great_dane/Great_dane_05337.jpg
I

Img: dogImages/train/007.American_foxhound/American_foxhound_00474.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07597.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03068.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04872.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02034.jpg
Img: dogImages/train/089.Irish_wolfhound/Irish_wolfhound_06036.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01038.jpg
Img: dogImages/train/037.Brittany/Brittany_02589.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03526.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03512.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02663.jpg
Img: dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00573.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05045.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02961.jpg
Img: dogImages/train/089.Irish_wolfhound/Irish_wolfhound

Img: dogImages/train/041.Bullmastiff/Bullmastiff_02895.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05087.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02160.jpg
Img: dogImages/train/037.Brittany/Brittany_02595.jpg
Img: dogImages/train/051.Chow_chow/Chow_chow_03630.jpg
Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01693.jpg
Img: dogImages/train/085.Irish_red_and_white_setter/Irish_red_and_white_setter_05774.jpg
Img: dogImages/train/093.Kerry_blue_terrier/Kerry_blue_terrier_06320.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07143.jpg
Img: dogImages/train/120.Pharaoh_hound/Pharaoh_hound_07739.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02794.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01256.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02763.jpg
Img: dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00586.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cav

Img: dogImages/train/041.Bullmastiff/Bullmastiff_02920.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08071.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04033.jpg
Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04528.jpg
Img: dogImages/train/058.Dandie_dinmont_terrier/Dandie_dinmont_terrier_04146.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04055.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01936.jpg
Img: dogImages/train/035.Boykin_spaniel/Boykin_spaniel_02494.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01337.jpg
Img: dogImages/train/040.Bulldog/Bulldog_02868.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00699.jpg
Img: dogImages/train/047.Chesapeake_bay_retriever/Chesapeake_bay_retriever_03376.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01255.jpg
Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01473.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01266.jpg
Img: dogImages/trai

Img: dogImages/train/050.Chinese_shar-pei/Chinese_shar-pei_03542.jpg
Img: dogImages/train/030.Border_terrier/Border_terrier_02104.jpg
Img: dogImages/train/001.Affenpinscher/Affenpinscher_00072.jpg
Img: dogImages/train/081.Greyhound/Greyhound_05535.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02985.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05090.jpg
Img: dogImages/train/056.Dachshund/Dachshund_03950.jpg
Img: dogImages/train/079.Great_pyrenees/Great_pyrenees_05405.jpg
Img: dogImages/train/052.Clumber_spaniel/Clumber_spaniel_03668.jpg
Img: dogImages/train/053.Cocker_spaniel/Cocker_spaniel_03750.jpg
Img: dogImages/train/034.Boxer/Boxer_02421.jpg
Img: dogImages/train/025.Black_and_tan_coonhound/Black_and_tan_coonhound_01777.jpg
Img: dogImages/train/022.Belgian_tervuren/Belgian_tervuren_01565.jpg
Img: dogImages/train/080.Greater_swiss_mountain_dog/Greater_swiss_mountain_dog_05463.jpg
Img: dogImages/train/034.Boxer/Boxer_02419.jpg
Img: dogImages/train/005.Alaska

Img: dogImages/train/091.Japanese_chin/Japanese_chin_06200.jpg
Img: dogImages/train/048.Chihuahua/Chihuahua_03454.jpg
Img: dogImages/train/002.Afghan_hound/Afghan_hound_00134.jpg
Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01652.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00721.jpg
Img: dogImages/train/126.Saint_bernard/Saint_bernard_08016.jpg
Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01627.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05059.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04583.jpg
Img: dogImages/train/035.Boykin_spaniel/Boykin_spaniel_02462.jpg
Img: dogImages/train/077.Gordon_setter/Gordon_setter_05264.jpg
Img: dogImages/train/006.American_eskimo_dog/American_eskimo_dog_00400.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04236.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07349.jpg


Img: dogImages/train/030.Border_terrier/Border_terrier_02094.jpg
Img: dogImages/train/120.Pharaoh_hound/Pharaoh_hound_07765.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02891.jpg
Img: dogImages/train/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05142.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01312.jpg
Img: dogImages/train/034.Boxer/Boxer_02378.jpg
Img: dogImages/train/069.French_bulldog/French_bulldog_04766.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01114.jpg
Img: dogImages/train/129.Tibetan_mastiff/Tibetan_mastiff_08142.jpg
Img: dogImages/train/030.Border_terrier/Border_terrier_02138.jpg
Img: dogImages/train/059.Doberman_pinscher/Doberman_pinscher_04173.jpg
Img: dogImages/train/002.Afghan_hound/Afghan_hound_00148.jpg
Img: dogImages/train/058.Dandie_dinmont_terrier/Dandie_dinmont_terrier_04099.jpg
Img: dogImages/train/123.Pomeranian/Pomeranian_07865.jpg
Img: dogImages/train/030.Border_terrier/Border_terrier_02137.jpg
Img: dogImages/train/050.Chinese_shar-p

Img: dogImages/train/105.Neapolitan_mastiff/Neapolitan_mastiff_06940.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05949.jpg
Img: dogImages/train/004.Akita/Akita_00290.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01737.jpg
Img: dogImages/train/130.Welsh_springer_spaniel/Welsh_springer_spaniel_08209.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04696.jpg
Img: dogImages/train/080.Greater_swiss_mountain_dog/Greater_swiss_mountain_dog_05449.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05745.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01841.jpg
Img: dogImages/train/071.German_shepherd_dog/German_shepherd_dog_04894.jpg
Img: dogImages/train/124.Poodle/Poodle_07956.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02084.jpg
Img: dogImages/train/130.Welsh_springer_spaniel/Welsh_springer_spaniel_08213.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08089.jpg
Img: dogImages/train/014

Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00797.jpg
Img: dogImages/train/111.Norwich_terrier/Norwich_terrier_07269.jpg
Img: dogImages/train/004.Akita/Akita_00232.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07323.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07298.jpg
Img: dogImages/train/099.Lhasa_apso/Lhasa_apso_06654.jpg
Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04527.jpg
Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00851.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05189.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05253.jpg
Img: dogImages/train/050.Chinese_shar-pei/Chinese_shar-pei_03567.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02787.jpg
Img: dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00610.jpg
Img: dogImages/train/095.

Img: dogImages/train/105.Neapolitan_mastiff/Neapolitan_mastiff_06946.jpg
Img: dogImages/train/047.Chesapeake_bay_retriever/Chesapeake_bay_retriever_03337.jpg
Img: dogImages/train/093.Kerry_blue_terrier/Kerry_blue_terrier_06311.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02212.jpg
Img: dogImages/train/064.English_toy_spaniel/English_toy_spaniel_04536.jpg
Img: dogImages/train/006.American_eskimo_dog/American_eskimo_dog_00456.jpg
Img: dogImages/train/091.Japanese_chin/Japanese_chin_06159.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06874.jpg
Img: dogImages/train/013.Australian_terrier/Australian_terrier_00939.jpg
Img: dogImages/train/119.Petit_basset_griffon_vendeen/Petit_basset_griffon_vendeen_07692.jpg
Img: dogImages/train/116.Parson_russell_terrier/Parson_russell_terrier_07519.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02293.jpg
Img: dogImages/train/044.Cane_corso/Cane_corso_03135.jpg
Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00857.jpg
Img: dogImag

Img: dogImages/train/088.Irish_water_spaniel/Irish_water_spaniel_05968.jpg
Img: dogImages/train/118.Pembroke_welsh_corgi/Pembroke_welsh_corgi_07626.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04297.jpg
Img: dogImages/train/048.Chihuahua/Chihuahua_03398.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07353.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03314.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04471.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06605.jpg
Img: dogImages/train/019.Bedlington_terrier/Bedlington_terrier_01344.jpg
Img: dogImages/train/062.English_setter/English_setter_04397.jpg
Img: dogImages/train/096.Labrador_retriever/Labrador_retriever_06458.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05944.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04346.jpg
Img: dogImages/train/131.Wirehaired_pointin

Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01430.jpg
Img: dogImages/train/110.Norwegian_lundehund/Norwegian_lundehund_07208.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05208.jpg
Img: dogImages/train/079.Great_pyrenees/Great_pyrenees_05393.jpg
Img: dogImages/train/085.Irish_red_and_white_setter/Irish_red_and_white_setter_05777.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01770.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04560.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00322.jpg
Img: dogImages/train/035.Boykin_spaniel/Boykin_spaniel_02489.jpg
Img: dogImages/train/095.Kuvasz/Kuvasz_06403.jpg
Img: dogImages/train/009.American_water_spaniel/American_water_spaniel_00629.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05901.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00758.jpg
Img: dogImages/train/082.Havanese/Havanese_05619.jpg
Img: dogImages/train/129.Tib

Img: dogImages/train/091.Japanese_chin/Japanese_chin_06192.jpg
Img: dogImages/train/025.Black_and_tan_coonhound/Black_and_tan_coonhound_01811.jpg
Img: dogImages/train/040.Bulldog/Bulldog_02831.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02029.jpg
Img: dogImages/train/002.Afghan_hound/Afghan_hound_00112.jpg
Img: dogImages/train/003.Airedale_terrier/Airedale_terrier_00209.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04445.jpg
Img: dogImages/train/044.Cane_corso/Cane_corso_03109.jpg
Img: dogImages/train/083.Ibizan_hound/Ibizan_hound_05663.jpg
Img: dogImages/train/028.Bluetick_coonhound/Bluetick_coonhound_01959.jpg
Img: dogImages/train/004.Akita/Akita_00294.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02899.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06118.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05730.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04476.jpg

Img: dogImages/train/045.Cardigan_welsh_corgi/Cardigan_welsh_corgi_03197.jpg
Img: dogImages/train/037.Brittany/Brittany_02643.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07177.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02242.jpg
Img: dogImages/train/004.Akita/Akita_00277.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04318.jpg
Img: dogImages/train/021.Belgian_sheepdog/Belgian_sheepdog_01556.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02296.jpg
Img: dogImages/train/081.Greyhound/Greyhound_05550.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01045.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07340.jpg
Img: dogImages/train/059.Doberman_pinscher/Doberman_pinscher_04160.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04447.jpg
Img: dogImages/train/089.Irish_wolfhound/Irish_wolfhound_06051.jpg
Img: dogImages/train/020.B

Img: dogImages/train/030.Border_terrier/Border_terrier_02119.jpg
Img: dogImages/train/003.Airedale_terrier/Airedale_terrier_00167.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02214.jpg
Img: dogImages/train/047.Chesapeake_bay_retriever/Chesapeake_bay_retriever_03334.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01870.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07565.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02087.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00374.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02731.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04598.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04038.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04087.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00770.jpg
Img: dogImages/train/099.Lhasa_apso/Lhasa_apso_06662.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06545.jpg
Img: 

Img: dogImages/train/013.Australian_terrier/Australian_terrier_00902.jpg
Img: dogImages/train/021.Belgian_sheepdog/Belgian_sheepdog_01500.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04017.jpg
Img: dogImages/train/019.Bedlington_terrier/Bedlington_terrier_01345.jpg
Img: dogImages/train/124.Poodle/Poodle_07941.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02654.jpg
Img: dogImages/train/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05154.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04880.jpg
Img: dogImages/train/072.German_shorthaired_pointer/German_shorthaired_pointer_04973.jpg
Img: dogImages/train/125.Portuguese_water_dog/Portuguese_water_dog_07987.jpg
Img: dogImages/train/001.Affenpinscher/Affenpinscher_00060.jpg
Img: dogImages/train/115.Papillon/Papillon_07452.jpg
Img: dogImages/train/094.Komondor/Komondor_06337.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01945.jpg
Img: dogImages/train/091.Japanese_chin/Japanese_chin_06219.jpg
Img: dogIma

Img: dogImages/train/096.Labrador_retriever/Labrador_retriever_06476.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00340.jpg
Img: dogImages/train/102.Manchester_terrier/Manchester_terrier_06781.jpg
Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01411.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01744.jpg
Img: dogImages/train/071.German_shepherd_dog/German_shepherd_dog_04954.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00347.jpg
Img: dogImages/train/020.Belgian_malinois/Belgian_malinois_01438.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02983.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01276.jpg
Img: dogImages/train/059.Doberman_pinscher/Doberman_pinscher_04170.jpg
Img: dogImages/train/034.Boxer/Boxer_02360.jpg
Img: dogImages/train/088.Irish_water_spaniel/Irish_water_spaniel_06000.jpg
Img: dogImages/train/025.Black_and_tan_coonhound/Black_and_tan_coonhound_01812.jpg
Img: dogImages/train/113.Old_english_sheepdog/Ol

Img: dogImages/train/019.Bedlington_terrier/Bedlington_terrier_01352.jpg
Img: dogImages/train/004.Akita/Akita_00288.jpg
Img: dogImages/train/124.Poodle/Poodle_07947.jpg
Img: dogImages/train/062.English_setter/English_setter_04381.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04588.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05939.jpg
Img: dogImages/train/056.Dachshund/Dachshund_04001.jpg
Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01688.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00372.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07136.jpg
Img: dogImages/train/048.Chihuahua/Chihuahua_03463.jpg
Img: dogImages/train/108.Norwegian_buhund/Norwegian_buhund_07106.jpg
Img: dogImages/train/044.Cane_corso/Cane_corso_03120.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07301.jpg
Img: dogImages/train/052.Clumber_spaniel/Clumber_spaniel_0

Img: dogImages/train/053.Cocker_spaniel/Cocker_spaniel_03778.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01697.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01830.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02271.jpg
Img: dogImages/train/106.Newfoundland/Newfoundland_07001.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04218.jpg
Img: dogImages/train/014.Basenji/Basenji_01027.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01110.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07384.jpg
Img: dogImages/train/092.Keeshond/Keeshond_06252.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03492.jpg
Img: dogImages/train/125.Portuguese_water_dog/Portuguese_water_dog_07962.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05259.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02353.jpg
Img: dogImages/train/014.Basenji/Basenji_00956.jpg
Img: dogImages/

Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06532.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terrier_02257.jpg
Img: dogImages/train/124.Poodle/Poodle_07960.jpg
Img: dogImages/train/085.Irish_red_and_white_setter/Irish_red_and_white_setter_05803.jpg
Img: dogImages/train/080.Greater_swiss_mountain_dog/Greater_swiss_mountain_dog_05441.jpg
Img: dogImages/train/052.Clumber_spaniel/Clumber_spaniel_03728.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04866.jpg
Img: dogImages/train/034.Boxer/Boxer_02401.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00379.jpg
Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00845.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01942.jpg
Img: dogImages/train/072.German_shorthaired_pointer/German_shorthaired_pointer_05013.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03507.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05119.jpg
Img: dogImages/train/063.English_s

Img: dogImages/train/001.Affenpinscher/Affenpinscher_00065.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06145.jpg
Img: dogImages/train/099.Lhasa_apso/Lhasa_apso_06622.jpg
Img: dogImages/train/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05150.jpg
Img: dogImages/train/119.Petit_basset_griffon_vendeen/Petit_basset_griffon_vendeen_07682.jpg
Img: dogImages/train/079.Great_pyrenees/Great_pyrenees_05369.jpg
Img: dogImages/train/036.Briard/Briard_02583.jpg
Img: dogImages/train/021.Belgian_sheepdog/Belgian_sheepdog_01525.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02892.jpg
Img: dogImages/train/100.Lowchen/Lowchen_06693.jpg
Img: dogImages/train/001.Affenpinscher/Affenpinscher_00069.jpg
Img: dogImages/train/054.Collie/Collie_03845.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02926.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02959.jpg
Img: dogImages/train/066.Field_spaniel/Field_spaniel_04638.jpg
Img: dogImages/train/046.Cavalier_king_charles_

Img: dogImages/train/022.Belgian_tervuren/Belgian_tervuren_01580.jpg
Img: dogImages/train/044.Cane_corso/Cane_corso_03134.jpg
Img: dogImages/train/063.English_springer_spaniel/English_springer_spaniel_04473.jpg
Img: dogImages/train/131.Wirehaired_pointing_griffon/Wirehaired_pointing_griffon_08272.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06575.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01743.jpg
Img: dogImages/train/018.Beauceron/Beauceron_01289.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02323.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01893.jpg
Img: dogImages/train/082.Havanese/Havanese_05567.jpg
Img: dogImages/train/100.Lowchen/Lowchen_06709.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01933.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04755.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01098.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03247

Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00389.jpg
Img: dogImages/train/049.Chinese_crested/Chinese_crested_03480.jpg
Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01658.jpg
Img: dogImages/train/066.Field_spaniel/Field_spaniel_04639.jpg
Img: dogImages/train/124.Poodle/Poodle_07954.jpg
Img: dogImages/train/132.Xoloitzcuintli/Xoloitzcuintli_08311.jpg
Img: dogImages/train/125.Portuguese_water_dog/Portuguese_water_dog_08000.jpg
Img: dogImages/train/130.Welsh_springer_spaniel/Welsh_springer_spaniel_08204.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07381.jpg
Img: dogImages/train/028.Bluetick_coonhound/Bluetick_coonhound_01987.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05929.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07592.jpg
Img: dogImages/train/065.Entlebucher_mountain_dog/Entlebucher_mountain_dog_04557.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_0728

Img: dogImages/train/029.Border_collie/Border_collie_02038.jpg
Img: dogImages/train/002.Afghan_hound/Afghan_hound_00126.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02335.jpg
Img: dogImages/train/021.Belgian_sheepdog/Belgian_sheepdog_01498.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02354.jpg
Img: dogImages/train/091.Japanese_chin/Japanese_chin_06160.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01773.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02909.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02680.jpg
Img: dogImages/train/039.Bull_terrier/Bull_terrier_02792.jpg
Img: dogImages/train/047.Chesapeake_bay_retriever/Chesapeake_bay_retriever_03378.jpg
Img: dogImages/train/004.Akita/Akita_00261.jpg
Img: dogImages/train/089.Irish_wolfhound/Irish_wolfhound_06027.jpg
Img: dogImages/train/116.Parson_russell_terrier/Parson_russell_terrier_07541.jpg
Img: dogImages/train/017.Bearded_collie/Bearded_collie_01261

Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02692.jpg
Img: dogImages/train/050.Chinese_shar-pei/Chinese_shar-pei_03581.jpg
Img: dogImages/train/004.Akita/Akita_00224.jpg
Img: dogImages/train/078.Great_dane/Great_dane_05338.jpg
Img: dogImages/train/007.American_foxhound/American_foxhound_00514.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06500.jpg
Img: dogImages/train/007.American_foxhound/American_foxhound_00518.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04273.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06562.jpg
Img: dogImages/train/123.Pomeranian/Pomeranian_07860.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04065.jpg
Img: dogImages/train/082.Havanese/Havanese_05583.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04253.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02998.jpg
Img: dogImages/train/013.Australian_terrier/Australian_terrier_00931.jpg
Img: dogImages/train/002.Afghan_hound/Afghan_ho

Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00301.jpg
Img: dogImages/train/115.Papillon/Papillon_07443.jpg
Img: dogImages/train/010.Anatolian_shepherd_dog/Anatolian_shepherd_dog_00680.jpg
Img: dogImages/train/048.Chihuahua/Chihuahua_03408.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03065.jpg
Img: dogImages/train/026.Black_russian_terrier/Black_russian_terrier_01826.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04836.jpg
Img: dogImages/train/034.Boxer/Boxer_02380.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05813.jpg
Img: dogImages/train/006.American_eskimo_dog/American_eskimo_dog_00453.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05809.jpg
Img: dogImages/train/037.Brittany/Brittany_02609.jpg
Img: dogImages/train/053.Cocker_spaniel/Cocker_spaniel_03736.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05222.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02165.jpg
Img: dogImages/train/082.Havanese/Havanese_05568.jpg
Img: d

Img: dogImages/train/030.Border_terrier/Border_terrier_02095.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06879.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02940.jpg
Img: dogImages/train/072.German_shorthaired_pointer/German_shorthaired_pointer_05002.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00764.jpg
Img: dogImages/train/024.Bichon_frise/Bichon_frise_01762.jpg
Img: dogImages/train/043.Canaan_dog/Canaan_dog_03094.jpg
Img: dogImages/train/057.Dalmatian/Dalmatian_04042.jpg
Img: dogImages/train/006.American_eskimo_dog/American_eskimo_dog_00468.jpg
Img: dogImages/train/054.Collie/Collie_03826.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04695.jpg
Img: dogImages/train/113.Old_english_sheepdog/Old_english_sheepdog_07354.jpg
Img: dogImages/train/111.Norwich_terrier/Norwich_terrier_07242.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05951.jpg
Img: dogImages/train/004.Akita/Akita_00297.jpg
Img: dogImages/train/023.Be

Img: dogImages/train/103.Mastiff/Mastiff_06851.jpg
Img: dogImages/train/025.Black_and_tan_coonhound/Black_and_tan_coonhound_01809.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05052.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06871.jpg
Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00807.jpg
Img: dogImages/train/089.Irish_wolfhound/Irish_wolfhound_06085.jpg
Img: dogImages/train/112.Nova_scotia_duck_tolling_retriever/Nova_scotia_duck_tolling_retriever_07284.jpg
Img: dogImages/train/083.Ibizan_hound/Ibizan_hound_05661.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02352.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06086.jpg
Img: dogImages/train/068.Flat-coated_retriever/Flat-coated_retriever_04756.jpg
Img: dogImages/train/124.Poodle/Poodle_07929.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05186.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charle

Img: dogImages/train/124.Poodle/Poodle_07902.jpg
Img: dogImages/train/123.Pomeranian/Pomeranian_07852.jpg
Img: dogImages/train/096.Labrador_retriever/Labrador_retriever_06489.jpg
Img: dogImages/train/083.Ibizan_hound/Ibizan_hound_05650.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07569.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02343.jpg
Img: dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03269.jpg
Img: dogImages/train/038.Brussels_griffon/Brussels_griffon_02673.jpg
Img: dogImages/train/014.Basenji/Basenji_01004.jpg
Img: dogImages/train/131.Wirehaired_pointing_griffon/Wirehaired_pointing_griffon_08269.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00780.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05739.jpg
Img: dogImages/train/117.Pekingese/Pekingese_07567.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02024.jpg
Img: dogImages/train/032.Boston_terrier/Boston_terri

Img: dogImages/train/040.Bulldog/Bulldog_02844.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08060.jpg
Img: dogImages/train/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05139.jpg
Img: dogImages/train/027.Bloodhound/Bloodhound_01938.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00771.jpg
Img: dogImages/train/019.Bedlington_terrier/Bedlington_terrier_01371.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05232.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00765.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01108.jpg
Img: dogImages/train/103.Mastiff/Mastiff_06875.jpg
Img: dogImages/train/091.Japanese_chin/Japanese_chin_06197.jpg
Img: dogImages/train/130.Welsh_springer_spaniel/Welsh_springer_spaniel_08239.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04308.jpg
Img: dogImages/train/014.Basenji/Basenji_00977.jpg
Img: dogImages/train/029.Border_collie/Border_collie_02050.jpg

Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04317.jpg
Img: dogImages/train/015.Basset_hound/Basset_hound_01088.jpg
Img: dogImages/train/066.Field_spaniel/Field_spaniel_04613.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00755.jpg
Img: dogImages/train/091.Japanese_chin/Japanese_chin_06165.jpg
Img: dogImages/train/092.Keeshond/Keeshond_06249.jpg
Img: dogImages/train/087.Irish_terrier/Irish_terrier_05908.jpg
Img: dogImages/train/070.German_pinscher/German_pinscher_04837.jpg
Img: dogImages/train/016.Beagle/Beagle_01131.jpg
Img: dogImages/train/051.Chow_chow/Chow_chow_03632.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02188.jpg
Img: dogImages/train/058.Dandie_dinmont_terrier/Dandie_dinmont_terrier_04139.jpg
Img: dogImages/train/036.Briard/Briard_02568.jpg
Img: dogImages/train/073.German_wirehaired_pointer/German_wirehaired_pointer_05063.jpg
Img: dogImages/train/071.German_shepherd_dog/German_shepherd_dog_04913.jpg
Img: dogImages/train/048.

Img: dogImages/train/012.Australian_shepherd/Australian_shepherd_00875.jpg
Img: dogImages/train/095.Kuvasz/Kuvasz_06434.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06132.jpg
Img: dogImages/train/072.German_shorthaired_pointer/German_shorthaired_pointer_04968.jpg
Img: dogImages/train/123.Pomeranian/Pomeranian_07851.jpg
Img: dogImages/train/131.Wirehaired_pointing_griffon/Wirehaired_pointing_griffon_08254.jpg
Img: dogImages/train/110.Norwegian_lundehund/Norwegian_lundehund_07207.jpg
Img: dogImages/train/035.Boykin_spaniel/Boykin_spaniel_02456.jpg
Img: dogImages/train/100.Lowchen/Lowchen_06683.jpg
Img: dogImages/train/040.Bulldog/Bulldog_02847.jpg
Img: dogImages/train/061.English_cocker_spaniel/English_cocker_spaniel_04305.jpg
Img: dogImages/train/101.Maltese/Maltese_06746.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvier_des_flandres_02337.jpg
Img: dogImages/train/042.Cairn_terrier/Cairn_terrier_02987.jpg
Img: dogImages/train/104.Miniature_schnauzer/Miniature

Img: dogImages/train/023.Bernese_mountain_dog/Bernese_mountain_dog_01640.jpg
Img: dogImages/train/084.Icelandic_sheepdog/Icelandic_sheepdog_05710.jpg
Img: dogImages/train/129.Tibetan_mastiff/Tibetan_mastiff_08136.jpg
Img: dogImages/train/090.Italian_greyhound/Italian_greyhound_06125.jpg
Img: dogImages/train/005.Alaskan_malamute/Alaskan_malamute_00351.jpg
Img: dogImages/train/093.Kerry_blue_terrier/Kerry_blue_terrier_06287.jpg
Img: dogImages/train/013.Australian_terrier/Australian_terrier_00917.jpg
Img: dogImages/train/048.Chihuahua/Chihuahua_03453.jpg
Img: dogImages/train/053.Cocker_spaniel/Cocker_spaniel_03751.jpg
Img: dogImages/train/034.Boxer/Boxer_02387.jpg
Img: dogImages/train/097.Lakeland_terrier/Lakeland_terrier_06553.jpg
Img: dogImages/train/109.Norwegian_elkhound/Norwegian_elkhound_07145.jpg
Img: dogImages/train/054.Collie/Collie_03857.jpg
Img: dogImages/train/019.Bedlington_terrier/Bedlington_terrier_01391.jpg
Img: dogImages/train/086.Irish_setter/Irish_setter_05853.jpg
Img: 

Img: dogImages/train/001.Affenpinscher/Affenpinscher_00027.jpg
Img: dogImages/train/004.Akita/Akita_00273.jpg
Img: dogImages/train/050.Chinese_shar-pei/Chinese_shar-pei_03578.jpg
Img: dogImages/train/034.Boxer/Boxer_02389.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06563.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05114.jpg
Img: dogImages/train/060.Dogue_de_bordeaux/Dogue_de_bordeaux_04280.jpg
Img: dogImages/train/122.Pointer/Pointer_07820.jpg
Img: dogImages/train/078.Great_dane/Great_dane_05333.jpg
Img: dogImages/train/004.Akita/Akita_00260.jpg
Img: dogImages/train/062.English_setter/English_setter_04393.jpg
Img: dogImages/train/041.Bullmastiff/Bullmastiff_02903.jpg
Img: dogImages/train/120.Pharaoh_hound/Pharaoh_hound_07735.jpg
Img: dogImages/train/011.Australian_cattle_dog/Australian_cattle_dog_00742.jpg
Img: dogImages/train/098.Leonberger/Leonberger_06594.jpg
Img: dogImages/train/031.Borzoi/Borzoi_02153.jpg
Img: dogImages/train/033.Bouvier_des_flandres/Bouvie

Img: dogImages/train/043.Canaan_dog/Canaan_dog_03057.jpg
Img: dogImages/train/116.Parson_russell_terrier/Parson_russell_terrier_07521.jpg
Img: dogImages/train/076.Golden_retriever/Golden_retriever_05233.jpg
Img: dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00542.jpg
Img: dogImages/train/056.Dachshund/Dachshund_03989.jpg
Img: dogImages/train/071.German_shepherd_dog/German_shepherd_dog_04930.jpg
Img: dogImages/train/127.Silky_terrier/Silky_terrier_08045.jpg
Img: dogImages/train/037.Brittany/Brittany_02631.jpg
Img: dogImages/train/072.German_shorthaired_pointer/German_shorthaired_pointer_05017.jpg
Img: dogImages/train/074.Giant_schnauzer/Giant_schnauzer_05091.jpg
Img: dogImages/train/102.Manchester_terrier/Manchester_terrier_06803.jpg
Img: dogImages/train/080.Greater_swiss_mountain_dog/Greater_swiss_mountain_dog_05444.jpg
Img: dogImages/train/092.Keeshond/Keeshond_06232.jpg
Img: dogImages/train/105.Neapolitan_mastiff/Neapolitan_mastiff_06955.jpg
Img: d

In [30]:
humans_X_short = humans_X[:200]
dogs_X_short = dogs_X[:200]

humans_y = []
for x in humans_X_short:
    humans_y.append([1, 0])
    
humans_y = np.array(humans_y, dtype=np.float32)

dogs_y = []
for x in dogs_X_short:
    dogs_y.append([0, 1])
dogs_y = np.array(dogs_y, dtype=np.float32)

dvh_X = np.concatenate((humans_X_short, dogs_X_short), axis=0)
dvh_y = np.concatenate((humans_y, dogs_y), axis=0)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(7, 7), input_shape=(224, 224, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(Conv2D(filters=32, kernel_size=(5, 5)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(120))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 200

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.dogs_vs_humans.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(dvh_X, dvh_y,
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 218, 218, 16)      2368      
_________________________________________________________________
activation_1 (Activation)    (None, 218, 218, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 105, 105, 32)      12832     
_________________________________________________________________
activation_2 (Activation)    (None, 105, 105, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 105, 105, 32)      0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
__________

/home/usuario/anaconda3/envs/aind-dog/lib/python3.6/site-packages/keras/callbacks.py:388: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/200
400/400 [==============================] - 18s - loss: 8.0590 - acc: 0.5000    
Epoch 3/200
400/400 [==============================] - 19s - loss: 8.0590 - acc: 0.5000    
Epoch 4/200
400/400 [==============================] - 19s - loss: 8.0590 - acc: 0.5000    
Epoch 5/200
400/400 [==============================] - 18s - loss: 8.0590 - acc: 0.5000    
Epoch 6/200
400/400 [==============================] - 19s - loss: 8.0590 - acc: 0.5000    
Epoch 7/200
400/400 [==============================] - 18s - loss: 8.0590 - acc: 0.5000    
Epoch 8/200
400/400 [==============================] - 18s - loss: 8.0590 - acc: 0.5000    
Epoch 9/200
400/400 [==============================] - 19s - loss: 8.0590 - acc: 0.5000    
Epoch 10/200
400/400 [==============================] - 20s - loss: 8.0590 - acc: 0.5000    
Epoch 11/200
400/400 [==============================] - 20s - loss: 8.0590 - acc: 0.5000    
Epoch 12/200
320/400 [=======================>......] - ETA: 3s - loss: 7.907

In [13]:
model.save_weights("saved_models/weights.best.dogs_vs_humans.hdf5")

NameError: name 'model' is not defined

In [90]:
humans_X_test = humans_X[100:200]
dogs_X_test = dogs_X[100:200]

my_X_test = np.concatenate((humans_X_test, dogs_X_test), axis=0)

my_predictions = [np.argmax(model.predict(np.expand_dims(t, axis=0))) for t in my_X_test]

dvh_test = []
for i in range(0, 100):
    dvh_test.append(0)

for i in range(0, 100):
    dvh_test.append(1)

# report test accuracy
my_test_accuracy = 100*np.sum(np.array(my_predictions)==np.array(dvh_test))/len(my_predictions)
print('Test accuracy: %.4f%%' % my_test_accuracy)

Test accuracy: 98.5000%


In [ ]:
"""
Answer 2:
A clearly presented face is not an acceptable premise in real-life scenarios. This can be obviated in many ways.
If utilizing haar-cascades, you can include other xml cascade files that also account for profile or skewed views, 
or even create your own, OpenCV has a train-cascade facility for that. However, this process is time consuming and not
nearly as accurate as constructing a deep learning network. In the above example, I built a deep neural network with
2 groups of CONV-RELU-POOL layers, a GAP layer to gather the best features and flatten the data, then two FC layers
with a softmax activation for the final classification. The example above is a proof-of-concept, run over a tiny 200
images dataset (100 humans and 100 dogs) and over only 200 epochs. This took roughly 2 hours to train and yielded and 
accuracy of 98.5%.
Out of curiosity, it seems the majority of the error lies in humans being mistaken for dogs.
"""

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a pre-trained [ResNet-50](http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006) model to detect dogs in images.  Our first line of code downloads the ResNet-50 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  Given an image, this pre-trained ResNet-50 model returns a prediction (derived from the available categories in ImageNet) for the object that is contained in the image.

In [7]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

### Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [8]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

### Making Predictions with ResNet-50

Getting the 4D tensor ready for ResNet-50, and for any other pre-trained model in Keras, requires some additional processing.  First, the RGB image is converted to BGR by reordering the channels.  All pre-trained models have the additional normalization step that the mean pixel (expressed in RGB as $[103.939, 116.779, 123.68]$ and calculated from all pixels in all images in ImageNet) must be subtracted from every pixel in each image.  This is implemented in the imported function `preprocess_input`.  If you're curious, you can check the code for `preprocess_input` [here](https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py).

Now that we have a way to format our image for supplying to ResNet-50, we are now ready to use the model to extract the predictions.  This is accomplished with the `predict` method, which returns an array whose $i$-th entry is the model's predicted probability that the image belongs to the $i$-th ImageNet category.  This is implemented in the `ResNet50_predict_labels` function below.

By taking the argmax of the predicted probability vector, we obtain an integer corresponding to the model's predicted object class, which we can identify with an object category through the use of this [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). 

In [9]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained ResNet-50 model, we need only check if the `ResNet50_predict_labels` function above returns a value between 151 and 268 (inclusive).

We use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [10]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### (IMPLEMENTATION) Assess the Dog Detector

__Question 3:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 

In [11]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
h_pc = len([h for h in human_files_short if dog_detector(h)])
d_pc = len([d for d in dog_files_short if dog_detector(d)])
print("Humans: {}%, dogs: {}%".format(h_pc * 1., d_pc * 1.))

Humans: 0.0%, dogs: 100.0%


---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 1%.  In Step 5 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

Be careful with adding too many trainable layers!  More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process.  Thankfully, Keras provides a handy estimate of the time that each epoch is likely to take; you can extrapolate this estimate to figure out how long it will take for your algorithm to train. 

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have great difficulty in distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun! 

### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [12]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:05<00:00, 141.14it/s]


### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        model.summary()

We have imported some Python modules to get you started, but feel free to import as many modules as you need.  If you end up getting stuck, here's a hint that specifies a model that trains relatively fast on CPU and attains >1% test accuracy in 5 epochs:

![Sample CNN](images/sample_cnn.png)
           
__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  If you chose to use the hinted architecture above, describe why you think that CNN architecture should work well for the image classification task.

__Answer:__ 

In [15]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(9,9), input_shape=(224, 224, 3)))
### TODO: Define your architecture.
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(Conv2D(filters=64, kernel_size=(7,7)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(500))
model.add(Activation("relu"))
model.add(Dense(133))
model.add(Activation("softmax"))
model.summary()

"""
Answer 4:
In the past I worked with well-know DL architectures, for example LeNet and AlexNet. They yielded satisfactory results
especially considering they are relatively shallow architectures. The architecture I adopted combines the suggested
architecture with a sort of LeNet, where the Flatten-ing happens through the GlobalAveragePooling, which I found to be
a lot more effective than a simple Flatten() as the features in the GAP level represent a feature.
In the first two CONV layers, I increase the filters as suggested by the lectures, but i reduce the kernel size as the
tensor size after a CONV-POOL layer reduces in height and width but it increases in depth.
I then feed the values coming from the GAP into two consecutive FC layers. This enables me to use at least one "relu"
activation layer in a FC layer, something I found to be very effective in the past. Ideally I would keep the three 
CONV-POOL sets of layers, but I sacrificed one for the sake of some training speed. Given some spare time, I would
plug the GAP + Dropout + FC + FC into the originally suggested architecture, which will result in a slower epoch pass
but higher accuracy.
"""

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 216, 216, 32)      7808      
_________________________________________________________________
activation_50 (Activation)   (None, 216, 216, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 108, 108, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 102, 102, 64)      100416    
_________________________________________________________________
activation_51 (Activation)   (None, 102, 102, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 102, 102, 64)      0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

'\nAnswer 4:\nIn the past I worked with well-know DL architectures, for example LeNet and AlexNet. They yielded satisfactory results\nespecially considering they are relatively shallow architectures. The architecture I adopted combines the suggested\narchitecture with a sort of LeNet, where the Flatten-ing happens through the GlobalAveragePooling, which I found to be\na lot more effective than a simple Flatten() as the features in the GAP level represent a feature.\nIn the first two CONV layers, I increase the filters as suggested by the lectures, but i reduce the kernel size as the\ntensor size after a CONV-POOL layer reduces in height and width but it increases in depth.\nI then feed the values coming from the GAP into two consecutive FC layers. This enables me to use at least one "relu"\nactivation layer in a FC layer, something I found to be very effective in the past. Ideally I would keep the three \nCONV-POOL sets of layers, but I sacrificed one for the sake of some training spee

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Compile the Model

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [19]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 10

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

RuntimeError: The model needs to be compiled before being used.

### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [15]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 6.6986%


---
<a id='step4'></a>
## Step 4: Use a CNN to Classify Dog Breeds

To reduce training time without sacrificing accuracy, we show you how to train a CNN using transfer learning.  In the following step, you will get a chance to use transfer learning to train your own CNN.

### Obtain Bottleneck Features

In [13]:
bottleneck_features = np.load('bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

### Model Architecture

The model uses the the pre-trained VGG-16 model as a fixed feature extractor, where the last convolutional output of VGG-16 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each dog category and is equipped with a softmax.

In [16]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))

VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 133)               68229     
Total params: 68,229.0
Trainable params: 68,229.0
Non-trainable params: 0.0
_________________________________________________________________


### Compile the Model

In [17]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

In [20]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 1s - loss: 11.7536 - acc: 0.1392 - val_loss: 10.1214 - val_acc: 0.2419
Epoch 2/20
6680/6680 [==============================] - 1s - loss: 9.3126 - acc: 0.3198 - val_loss: 9.3895 - val_acc: 0.3150
Epoch 3/20
6680/6680 [==============================] - 1s - loss: 8.8248 - acc: 0.3895 - val_loss: 9.2807 - val_acc: 0.3377
Epoch 4/20
6680/6680 [==============================] - 1s - loss: 8.6324 - acc: 0.4187 - val_loss: 9.0734 - val_acc: 0.3521
Epoch 5/20
6680/6680 [==============================] - 1s - loss: 8.4752 - acc: 0.4380 - val_loss: 9.0898 - val_acc: 0.3689
Epoch 6/20
6680/6680 [==============================] - 1s - loss: 8.3793 - acc: 0.4542 - val_loss: 8.9809 - val_acc: 0.3713
Epoch 7/20
6680/6680 [==============================] - 1s - loss: 8.3339 - acc: 0.4641 - val_loss: 8.8840 - val_acc: 0.3808
Epoch 8/20
6680/6680 [==============================] - 1s - loss: 8.2208 - 

### Load the Model with the Best Validation Loss

In [21]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

Now, we can use the CNN to test how well it identifies breed within our test dataset of dog images.  We print the test accuracy below.

In [22]:
# get index of predicted dog breed for each image in test set
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 45.3349%


### Predict Dog Breed with the Model

In [23]:
from extract_bottleneck_features import *

def VGG16_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

---
<a id='step5'></a>
## Step 5: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

In Step 4, we used transfer learning to create a CNN using VGG-16 bottleneck features.  In this section, you must use the bottleneck features from a different pre-trained model.  To make things easier for you, we have pre-computed the features for all of the networks that are currently available in Keras:
- [VGG-19](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogVGG19Data.npz) bottleneck features
- [ResNet-50](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz) bottleneck features
- [Inception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz) bottleneck features
- [Xception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogXceptionData.npz) bottleneck features

The files are encoded as such:

    Dog{network}Data.npz
    
where `{network}`, in the above filename, can be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.  Pick one of the above architectures, download the corresponding bottleneck features, and store the downloaded file in the `bottleneck_features/` folder in the repository.

### (IMPLEMENTATION) Obtain Bottleneck Features

In the code block below, extract the bottleneck features corresponding to the train, test, and validation sets by running the following:

    bottleneck_features = np.load('bottleneck_features/Dog{network}Data.npz')
    train_{network} = bottleneck_features['train']
    valid_{network} = bottleneck_features['valid']
    test_{network} = bottleneck_features['test']

In [25]:
### TODO: Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('bottleneck_features/DogInceptionV3Data.npz')
train_Inception = bottleneck_features['train']
valid_Inception = bottleneck_features['valid']
test_Inception = bottleneck_features['test']

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        <your model's name>.summary()
   
__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 



In [26]:
### TODO: Define your architecture.
"""
The architecture of my network is defined as follows: a first GAP layer to extract the best features from the images
as processed by the Inception bottleneck features, which is then fed into a 500-nodes FC layer 
activated with tanh and a final 133-nodes FC layer with softmax for final classification.
I applied a 20% dropout probability before the final FC layer.

Reasoning
---------
The original architecture (InceptionV3) applies numerous CONV and POOL filters, which made me think there was no more 
need for further convolutions to be applied in my network.
Indeed adding more CONV-POOL lauers seemed to slow the training down to an impractical level, and I could not see
enough improvements made in loss and accuracy. I might have been doing something wrong but I had to interrupt the
training and revise the architecture.

Instead, I wanted to squeeze the best features processed through Inception weights, 
and feed them into a non-linear neural network. I find a GAP layer ideal for the job for 
two purposes: a. it retrieves the best features from an image processed through Inception weights, and b. it flattens
the data as a bonus. It's like a non-naive Flatten(). From there, as I was facing a non-linear problem, I needed at
at least another 2 FC layers. My reasoning for choosing tanh over relu or sigmoid is pure empirical, after a few tries, 
I found that tanh was performing better than other activation functions in this particular case. I added a dropout layer
to improve accuracy. Again, empirically I found 0.2 to be the best value, with 0.1 not making too great a difference, 
while anything bigger than 0.2 worsening the model even significantly. In the past I used GridSearchCV in scikit-learn
to perform hyper parameters tuning with good results, given enough time I'd like to try this out for Dropout value, 
activation function for the first FC layer and number of nodes in the first FC layer.

-----------------------------

The choice of model (InceptionV3) for transfer learning was an empirical one. VGG19 performed around 67% accuracy, ResNet around 80% 
while Inception yielded close to 82% (81.8) which is quite impressive.
"""
Inc_model = Sequential()
Inc_model.add(GlobalAveragePooling2D(input_shape=train_Inception.shape[1:]))
Inc_model.add(Dense(500))
Inc_model.add(Activation("tanh"))
Inc_model.add(Dropout(0.2))
Inc_model.add(Dense(133, activation='softmax'))
Inc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               1024500   
_________________________________________________________________
activation_54 (Activation)   (None, 500)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 133)               66633     
Total params: 1,091,133.0
Trainable params: 1,091,133.0
Non-trainable params: 0.0
_________________________________________________________________


### (IMPLEMENTATION) Compile the Model

In [28]:
### TODO: Compile the model.
Inc_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.  

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [31]:
### TODO: Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Inc_model.hdf5', 
                               verbose=1, save_best_only=True)

Inc_model.fit(train_Inception, train_targets, 
          validation_data=(valid_Inception, valid_targets),
          epochs=2000, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/2000
6680/6680 [==============================] - 6s - loss: 0.5489 - acc: 0.8362 - val_loss: 0.6992 - val_acc: 0.8024
Epoch 2/2000
6680/6680 [==============================] - 6s - loss: 0.4344 - acc: 0.8605 - val_loss: 0.6161 - val_acc: 0.8096
Epoch 3/2000
6680/6680 [==============================] - 6s - loss: 0.3600 - acc: 0.8855 - val_loss: 0.5951 - val_acc: 0.8431
Epoch 4/2000
6680/6680 [==============================] - 6s - loss: 0.2976 - acc: 0.9105 - val_loss: 0.6691 - val_acc: 0.8275
Epoch 5/2000
6680/6680 [==============================] - 6s - loss: 0.2592 - acc: 0.9195 - val_loss: 0.5674 - val_acc: 0.8419
Epoch 6/2000
6680/6680 [==============================] - 6s - loss: 0.2197 - acc: 0.9320 - val_loss: 0.5851 - val_acc: 0.8395
Epoch 7/2000
6680/6680 [==============================] - 6s - loss: 0.1883 - acc: 0.9418 - val_loss: 0.6489 - val_acc: 0.8371
Epoch 8/2000
6680/6680 [==============================] - 6s - l

6680/6680 [==============================] - 6s - loss: 0.0142 - acc: 0.9978 - val_loss: 0.9651 - val_acc: 0.8563
Epoch 66/2000
6680/6680 [==============================] - 6s - loss: 0.0127 - acc: 0.9970 - val_loss: 1.0517 - val_acc: 0.8515
Epoch 67/2000
6680/6680 [==============================] - 6s - loss: 0.0146 - acc: 0.9976 - val_loss: 1.0567 - val_acc: 0.8515
Epoch 68/2000
6680/6680 [==============================] - 6s - loss: 0.0127 - acc: 0.9978 - val_loss: 1.0249 - val_acc: 0.8395
Epoch 69/2000
6680/6680 [==============================] - 6s - loss: 0.0107 - acc: 0.9978 - val_loss: 1.0196 - val_acc: 0.8515
Epoch 70/2000
6680/6680 [==============================] - 6s - loss: 0.0144 - acc: 0.9966 - val_loss: 1.0185 - val_acc: 0.8503
Epoch 71/2000
6680/6680 [==============================] - 6s - loss: 0.0135 - acc: 0.9963 - val_loss: 0.9952 - val_acc: 0.8587
Epoch 72/2000
6680/6680 [==============================] - 6s - loss: 0.0156 - acc: 0.9973 - val_loss: 0.9708 - val_ac

6680/6680 [==============================] - 6s - loss: 0.0110 - acc: 0.9979 - val_loss: 1.2493 - val_acc: 0.8455
Epoch 132/2000
6680/6680 [==============================] - 6s - loss: 0.0077 - acc: 0.9985 - val_loss: 1.0761 - val_acc: 0.8659
Epoch 133/2000
6680/6680 [==============================] - 6s - loss: 0.0091 - acc: 0.9978 - val_loss: 1.2033 - val_acc: 0.8491
Epoch 134/2000
6680/6680 [==============================] - 6s - loss: 0.0110 - acc: 0.9981 - val_loss: 1.1951 - val_acc: 0.8443
Epoch 135/2000
6680/6680 [==============================] - 6s - loss: 0.0132 - acc: 0.9975 - val_loss: 1.1978 - val_acc: 0.8479
Epoch 136/2000
6680/6680 [==============================] - 6s - loss: 0.0112 - acc: 0.9976 - val_loss: 1.2009 - val_acc: 0.8623
Epoch 137/2000
6680/6680 [==============================] - 6s - loss: 0.0115 - acc: 0.9976 - val_loss: 1.2285 - val_acc: 0.8515
Epoch 138/2000
6680/6680 [==============================] - 6s - loss: 0.0128 - acc: 0.9984 - val_loss: 1.2153 -

6680/6680 [==============================] - 6s - loss: 0.0069 - acc: 0.9982 - val_loss: 1.3175 - val_acc: 0.8515
Epoch 198/2000
6680/6680 [==============================] - 6s - loss: 0.0115 - acc: 0.9978 - val_loss: 1.3237 - val_acc: 0.8443
Epoch 199/2000
6680/6680 [==============================] - 6s - loss: 0.0106 - acc: 0.9979 - val_loss: 1.3198 - val_acc: 0.8503
Epoch 200/2000
6680/6680 [==============================] - 6s - loss: 0.0154 - acc: 0.9975 - val_loss: 1.3083 - val_acc: 0.8515
Epoch 201/2000
6680/6680 [==============================] - 6s - loss: 0.0076 - acc: 0.9984 - val_loss: 1.3300 - val_acc: 0.8395
Epoch 202/2000
6680/6680 [==============================] - 6s - loss: 0.0099 - acc: 0.9985 - val_loss: 1.2509 - val_acc: 0.8527
Epoch 203/2000
6680/6680 [==============================] - 6s - loss: 0.0147 - acc: 0.9979 - val_loss: 1.2625 - val_acc: 0.8551
Epoch 204/2000
6680/6680 [==============================] - 6s - loss: 0.0128 - acc: 0.9978 - val_loss: 1.2818 -

6680/6680 [==============================] - 6s - loss: 0.0148 - acc: 0.9979 - val_loss: 1.3761 - val_acc: 0.8623
Epoch 264/2000
6680/6680 [==============================] - 6s - loss: 0.0127 - acc: 0.9978 - val_loss: 1.4139 - val_acc: 0.8443
Epoch 265/2000
6680/6680 [==============================] - 6s - loss: 0.0129 - acc: 0.9982 - val_loss: 1.3802 - val_acc: 0.8563
Epoch 266/2000
6680/6680 [==============================] - 6s - loss: 0.0107 - acc: 0.9973 - val_loss: 1.3703 - val_acc: 0.8455
Epoch 267/2000
6680/6680 [==============================] - 6s - loss: 0.0115 - acc: 0.9984 - val_loss: 1.3350 - val_acc: 0.8623
Epoch 268/2000
6680/6680 [==============================] - 6s - loss: 0.0113 - acc: 0.9981 - val_loss: 1.4452 - val_acc: 0.8551
Epoch 269/2000
6680/6680 [==============================] - 6s - loss: 0.0066 - acc: 0.9987 - val_loss: 1.4333 - val_acc: 0.8491
Epoch 270/2000
6680/6680 [==============================] - 6s - loss: 0.0113 - acc: 0.9984 - val_loss: 1.3889 -

6680/6680 [==============================] - 7s - loss: 0.0118 - acc: 0.9985 - val_loss: 1.4781 - val_acc: 0.8455
Epoch 330/2000
6680/6680 [==============================] - 7s - loss: 0.0136 - acc: 0.9984 - val_loss: 1.3632 - val_acc: 0.8563
Epoch 331/2000
6680/6680 [==============================] - 7s - loss: 0.0069 - acc: 0.9988 - val_loss: 1.4199 - val_acc: 0.8479
Epoch 332/2000
6680/6680 [==============================] - 6s - loss: 0.0084 - acc: 0.9984 - val_loss: 1.4528 - val_acc: 0.8539
Epoch 333/2000
6680/6680 [==============================] - 7s - loss: 0.0135 - acc: 0.9982 - val_loss: 1.4081 - val_acc: 0.8479
Epoch 334/2000
6680/6680 [==============================] - 6s - loss: 0.0118 - acc: 0.9985 - val_loss: 1.4933 - val_acc: 0.8443
Epoch 335/2000
6680/6680 [==============================] - 7s - loss: 0.0095 - acc: 0.9981 - val_loss: 1.5236 - val_acc: 0.8419
Epoch 336/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9984 - val_loss: 1.4680 -

6680/6680 [==============================] - 7s - loss: 0.0139 - acc: 0.9982 - val_loss: 1.4443 - val_acc: 0.8467
Epoch 396/2000
6680/6680 [==============================] - 7s - loss: 0.0159 - acc: 0.9985 - val_loss: 1.4602 - val_acc: 0.8407
Epoch 397/2000
6680/6680 [==============================] - 7s - loss: 0.0096 - acc: 0.9985 - val_loss: 1.3853 - val_acc: 0.8563
Epoch 398/2000
6680/6680 [==============================] - 7s - loss: 0.0086 - acc: 0.9987 - val_loss: 1.4559 - val_acc: 0.8611
Epoch 399/2000
6680/6680 [==============================] - 7s - loss: 0.0122 - acc: 0.9985 - val_loss: 1.5154 - val_acc: 0.8479
Epoch 400/2000
6680/6680 [==============================] - 7s - loss: 0.0136 - acc: 0.9984 - val_loss: 1.4578 - val_acc: 0.8611
Epoch 401/2000
6680/6680 [==============================] - 7s - loss: 0.0096 - acc: 0.9988 - val_loss: 1.5020 - val_acc: 0.8479
Epoch 402/2000
6680/6680 [==============================] - 7s - loss: 0.0123 - acc: 0.9990 - val_loss: 1.4423 -

6680/6680 [==============================] - 7s - loss: 0.0144 - acc: 0.9987 - val_loss: 1.5199 - val_acc: 0.8407
Epoch 461/2000
6680/6680 [==============================] - 7s - loss: 0.0063 - acc: 0.9991 - val_loss: 1.5078 - val_acc: 0.8491
Epoch 462/2000
6680/6680 [==============================] - 7s - loss: 0.0088 - acc: 0.9991 - val_loss: 1.5280 - val_acc: 0.8503
Epoch 463/2000
6680/6680 [==============================] - 7s - loss: 0.0143 - acc: 0.9987 - val_loss: 1.4613 - val_acc: 0.8539
Epoch 464/2000
6680/6680 [==============================] - 7s - loss: 0.0130 - acc: 0.9985 - val_loss: 1.4843 - val_acc: 0.8515
Epoch 465/2000
6680/6680 [==============================] - 7s - loss: 0.0092 - acc: 0.9990 - val_loss: 1.4431 - val_acc: 0.8467
Epoch 466/2000
6680/6680 [==============================] - 7s - loss: 0.0137 - acc: 0.9984 - val_loss: 1.4816 - val_acc: 0.8467
Epoch 467/2000
6680/6680 [==============================] - 7s - loss: 0.0147 - acc: 0.9988 - val_loss: 1.5328 -

6680/6680 [==============================] - 7s - loss: 0.0123 - acc: 0.9988 - val_loss: 1.4369 - val_acc: 0.8599
Epoch 526/2000
6680/6680 [==============================] - 7s - loss: 0.0141 - acc: 0.9981 - val_loss: 1.4301 - val_acc: 0.8455
Epoch 527/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9990 - val_loss: 1.4316 - val_acc: 0.8659
Epoch 528/2000
6680/6680 [==============================] - 7s - loss: 0.0124 - acc: 0.9990 - val_loss: 1.3541 - val_acc: 0.8611
Epoch 529/2000
6680/6680 [==============================] - 7s - loss: 0.0128 - acc: 0.9987 - val_loss: 1.4927 - val_acc: 0.8515
Epoch 530/2000
6680/6680 [==============================] - 7s - loss: 0.0097 - acc: 0.9987 - val_loss: 1.4818 - val_acc: 0.8587
Epoch 531/2000
6680/6680 [==============================] - 7s - loss: 0.0137 - acc: 0.9984 - val_loss: 1.4750 - val_acc: 0.8527
Epoch 532/2000
6680/6680 [==============================] - 7s - loss: 0.0141 - acc: 0.9988 - val_loss: 1.4371 -

6680/6680 [==============================] - 7s - loss: 0.0097 - acc: 0.9991 - val_loss: 1.6581 - val_acc: 0.8431
Epoch 592/2000
6680/6680 [==============================] - 7s - loss: 0.0066 - acc: 0.9994 - val_loss: 1.5765 - val_acc: 0.8479
Epoch 593/2000
6680/6680 [==============================] - 7s - loss: 0.0158 - acc: 0.9990 - val_loss: 1.6160 - val_acc: 0.8407
Epoch 594/2000
6680/6680 [==============================] - 7s - loss: 0.0155 - acc: 0.9982 - val_loss: 1.5222 - val_acc: 0.8491
Epoch 595/2000
6680/6680 [==============================] - 7s - loss: 0.0090 - acc: 0.9991 - val_loss: 1.5063 - val_acc: 0.8455
Epoch 596/2000
6680/6680 [==============================] - 7s - loss: 0.0080 - acc: 0.9988 - val_loss: 1.5219 - val_acc: 0.8431
Epoch 597/2000
6680/6680 [==============================] - 7s - loss: 0.0148 - acc: 0.9990 - val_loss: 1.5474 - val_acc: 0.8503
Epoch 598/2000
6680/6680 [==============================] - 7s - loss: 0.0130 - acc: 0.9985 - val_loss: 1.6034 -

6680/6680 [==============================] - 7s - loss: 0.0159 - acc: 0.9979 - val_loss: 1.5311 - val_acc: 0.8455
Epoch 656/2000
6680/6680 [==============================] - 7s - loss: 0.0092 - acc: 0.9993 - val_loss: 1.5721 - val_acc: 0.8419
Epoch 657/2000
6680/6680 [==============================] - 7s - loss: 0.0086 - acc: 0.9991 - val_loss: 1.4903 - val_acc: 0.8623
Epoch 658/2000
6680/6680 [==============================] - 7s - loss: 0.0116 - acc: 0.9982 - val_loss: 1.4804 - val_acc: 0.8515
Epoch 659/2000
6680/6680 [==============================] - 7s - loss: 0.0100 - acc: 0.9990 - val_loss: 1.5511 - val_acc: 0.8563
Epoch 660/2000
6680/6680 [==============================] - 7s - loss: 0.0159 - acc: 0.9985 - val_loss: 1.6181 - val_acc: 0.8479
Epoch 661/2000
6680/6680 [==============================] - 7s - loss: 0.0104 - acc: 0.9990 - val_loss: 1.5570 - val_acc: 0.8515
Epoch 662/2000
6680/6680 [==============================] - 7s - loss: 0.0132 - acc: 0.9987 - val_loss: 1.5723 -

6680/6680 [==============================] - 7s - loss: 0.0140 - acc: 0.9990 - val_loss: 1.6077 - val_acc: 0.8539
Epoch 721/2000
6680/6680 [==============================] - 7s - loss: 0.0055 - acc: 0.9994 - val_loss: 1.6385 - val_acc: 0.8479
Epoch 722/2000
6680/6680 [==============================] - 7s - loss: 0.0105 - acc: 0.9993 - val_loss: 1.5898 - val_acc: 0.8515
Epoch 723/2000
6680/6680 [==============================] - 7s - loss: 0.0151 - acc: 0.9990 - val_loss: 1.5296 - val_acc: 0.8455
Epoch 724/2000
6680/6680 [==============================] - 7s - loss: 0.0146 - acc: 0.9988 - val_loss: 1.5413 - val_acc: 0.8431
Epoch 725/2000
6680/6680 [==============================] - 7s - loss: 0.0088 - acc: 0.9991 - val_loss: 1.6304 - val_acc: 0.8467
Epoch 726/2000
6680/6680 [==============================] - 7s - loss: 0.0100 - acc: 0.9990 - val_loss: 1.5952 - val_acc: 0.8539
Epoch 727/2000
6680/6680 [==============================] - 7s - loss: 0.0113 - acc: 0.9984 - val_loss: 1.5942 -

6680/6680 [==============================] - 7s - loss: 0.0167 - acc: 0.9985 - val_loss: 1.5953 - val_acc: 0.8551
Epoch 786/2000
6680/6680 [==============================] - 7s - loss: 0.0138 - acc: 0.9990 - val_loss: 1.5015 - val_acc: 0.8539
Epoch 787/2000
6680/6680 [==============================] - 7s - loss: 0.0126 - acc: 0.9988 - val_loss: 1.6530 - val_acc: 0.8407
Epoch 788/2000
6680/6680 [==============================] - 7s - loss: 0.0147 - acc: 0.9985 - val_loss: 1.5476 - val_acc: 0.8659
Epoch 789/2000
6680/6680 [==============================] - 7s - loss: 0.0125 - acc: 0.9988 - val_loss: 1.6350 - val_acc: 0.8467
Epoch 790/2000
6680/6680 [==============================] - 7s - loss: 0.0093 - acc: 0.9991 - val_loss: 1.5248 - val_acc: 0.8539
Epoch 791/2000
6680/6680 [==============================] - 7s - loss: 0.0117 - acc: 0.9991 - val_loss: 1.5554 - val_acc: 0.8623
Epoch 792/2000
6680/6680 [==============================] - 7s - loss: 0.0162 - acc: 0.9988 - val_loss: 1.5913 -

6680/6680 [==============================] - 7s - loss: 0.0115 - acc: 0.9990 - val_loss: 1.5596 - val_acc: 0.8611
Epoch 851/2000
6680/6680 [==============================] - 7s - loss: 0.0124 - acc: 0.9988 - val_loss: 1.5588 - val_acc: 0.8563
Epoch 852/2000
6680/6680 [==============================] - 7s - loss: 0.0148 - acc: 0.9984 - val_loss: 1.4380 - val_acc: 0.8695
Epoch 853/2000
6680/6680 [==============================] - 7s - loss: 0.0140 - acc: 0.9988 - val_loss: 1.5536 - val_acc: 0.8491
Epoch 854/2000
6680/6680 [==============================] - 7s - loss: 0.0144 - acc: 0.9987 - val_loss: 1.5609 - val_acc: 0.8491
Epoch 855/2000
6680/6680 [==============================] - 7s - loss: 0.0126 - acc: 0.9991 - val_loss: 1.5766 - val_acc: 0.8611
Epoch 856/2000
6680/6680 [==============================] - 7s - loss: 0.0150 - acc: 0.9987 - val_loss: 1.5355 - val_acc: 0.8527
Epoch 857/2000
6680/6680 [==============================] - 7s - loss: 0.0105 - acc: 0.9990 - val_loss: 1.5827 -

6680/6680 [==============================] - 7s - loss: 0.0076 - acc: 0.9991 - val_loss: 1.6257 - val_acc: 0.8431
Epoch 916/2000
6680/6680 [==============================] - 7s - loss: 0.0140 - acc: 0.9988 - val_loss: 1.5663 - val_acc: 0.8515
Epoch 917/2000
6680/6680 [==============================] - 7s - loss: 0.0126 - acc: 0.9990 - val_loss: 1.5731 - val_acc: 0.8491
Epoch 918/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9988 - val_loss: 1.5361 - val_acc: 0.8467
Epoch 919/2000
6680/6680 [==============================] - 7s - loss: 0.0109 - acc: 0.9988 - val_loss: 1.5756 - val_acc: 0.8551
Epoch 920/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9990 - val_loss: 1.5896 - val_acc: 0.8527
Epoch 921/2000
6680/6680 [==============================] - 8s - loss: 0.0109 - acc: 0.9987 - val_loss: 1.5859 - val_acc: 0.8443
Epoch 922/2000
6680/6680 [==============================] - 7s - loss: 0.0147 - acc: 0.9988 - val_loss: 1.7120 -

6680/6680 [==============================] - 7s - loss: 0.0103 - acc: 0.9993 - val_loss: 1.5850 - val_acc: 0.8635
Epoch 981/2000
6680/6680 [==============================] - 7s - loss: 0.0091 - acc: 0.9991 - val_loss: 1.6641 - val_acc: 0.8527
Epoch 982/2000
6680/6680 [==============================] - 7s - loss: 0.0146 - acc: 0.9988 - val_loss: 1.5702 - val_acc: 0.8599
Epoch 983/2000
6680/6680 [==============================] - 7s - loss: 0.0130 - acc: 0.9990 - val_loss: 1.6164 - val_acc: 0.8527
Epoch 984/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9987 - val_loss: 1.5470 - val_acc: 0.8623
Epoch 985/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9990 - val_loss: 1.6925 - val_acc: 0.8383
Epoch 986/2000
6680/6680 [==============================] - 7s - loss: 0.0153 - acc: 0.9990 - val_loss: 1.5761 - val_acc: 0.8563
Epoch 987/2000
6680/6680 [==============================] - 7s - loss: 0.0111 - acc: 0.9993 - val_loss: 1.6089 -

6680/6680 [==============================] - 8s - loss: 0.0125 - acc: 0.9988 - val_loss: 1.6302 - val_acc: 0.8539
Epoch 1014/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9990 - val_loss: 1.6161 - val_acc: 0.8503
Epoch 1015/2000
6680/6680 [==============================] - 7s - loss: 0.0109 - acc: 0.9991 - val_loss: 1.6599 - val_acc: 0.8515
Epoch 1016/2000
6680/6680 [==============================] - 7s - loss: 0.0101 - acc: 0.9991 - val_loss: 1.6836 - val_acc: 0.8467
Epoch 1017/2000
6680/6680 [==============================] - 7s - loss: 0.0165 - acc: 0.9982 - val_loss: 1.6120 - val_acc: 0.8419
Epoch 1018/2000
6680/6680 [==============================] - 8s - loss: 0.0160 - acc: 0.9985 - val_loss: 1.5835 - val_acc: 0.8551
Epoch 1019/2000
6680/6680 [==============================] - 8s - loss: 0.0077 - acc: 0.9991 - val_loss: 1.6361 - val_acc: 0.8407
Epoch 1020/2000
6680/6680 [==============================] - 8s - loss: 0.0098 - acc: 0.9993 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0130 - acc: 0.9990 - val_loss: 1.5640 - val_acc: 0.8479
Epoch 1047/2000
6680/6680 [==============================] - 8s - loss: 0.0108 - acc: 0.9991 - val_loss: 1.5998 - val_acc: 0.8455
Epoch 1048/2000
6680/6680 [==============================] - 8s - loss: 0.0124 - acc: 0.9991 - val_loss: 1.5982 - val_acc: 0.8563
Epoch 1049/2000
6680/6680 [==============================] - 8s - loss: 0.0139 - acc: 0.9988 - val_loss: 1.6096 - val_acc: 0.8515
Epoch 1050/2000
6680/6680 [==============================] - 8s - loss: 0.0130 - acc: 0.9991 - val_loss: 1.5388 - val_acc: 0.8611
Epoch 1051/2000
6680/6680 [==============================] - 8s - loss: 0.0149 - acc: 0.9987 - val_loss: 1.5891 - val_acc: 0.8503
Epoch 1052/2000
6680/6680 [==============================] - 8s - loss: 0.0108 - acc: 0.9991 - val_loss: 1.6290 - val_acc: 0.8575
Epoch 1053/2000
6680/6680 [==============================] - 8s - loss: 0.0089 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0115 - acc: 0.9991 - val_loss: 1.6488 - val_acc: 0.8527
Epoch 1112/2000
6680/6680 [==============================] - 8s - loss: 0.0139 - acc: 0.9991 - val_loss: 1.6069 - val_acc: 0.8503
Epoch 1113/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9993 - val_loss: 1.6603 - val_acc: 0.8551
Epoch 1114/2000
6680/6680 [==============================] - 8s - loss: 0.0106 - acc: 0.9990 - val_loss: 1.7039 - val_acc: 0.8587
Epoch 1115/2000
6680/6680 [==============================] - 8s - loss: 0.0109 - acc: 0.9993 - val_loss: 1.5970 - val_acc: 0.8515
Epoch 1116/2000
6680/6680 [==============================] - 8s - loss: 0.0124 - acc: 0.9991 - val_loss: 1.5856 - val_acc: 0.8479
Epoch 1117/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9993 - val_loss: 1.6591 - val_acc: 0.8503
Epoch 1118/2000
6680/6680 [==============================] - 8s - loss: 0.0122 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 7s - loss: 0.0103 - acc: 0.9990 - val_loss: 1.7064 - val_acc: 0.8407
Epoch 1177/2000
6680/6680 [==============================] - 8s - loss: 0.0131 - acc: 0.9991 - val_loss: 1.5810 - val_acc: 0.8407
Epoch 1178/2000
6680/6680 [==============================] - 7s - loss: 0.0120 - acc: 0.9991 - val_loss: 1.6838 - val_acc: 0.8491
Epoch 1179/2000
6680/6680 [==============================] - 7s - loss: 0.0134 - acc: 0.9988 - val_loss: 1.6483 - val_acc: 0.8575
Epoch 1180/2000
6680/6680 [==============================] - 8s - loss: 0.0140 - acc: 0.9990 - val_loss: 1.6334 - val_acc: 0.8443
Epoch 1181/2000
6680/6680 [==============================] - 8s - loss: 0.0103 - acc: 0.9993 - val_loss: 1.6470 - val_acc: 0.8479
Epoch 1182/2000
6680/6680 [==============================] - 8s - loss: 0.0116 - acc: 0.9993 - val_loss: 1.6629 - val_acc: 0.8539
Epoch 1183/2000
6680/6680 [==============================] - 7s - loss: 0.0085 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6153 - val_acc: 0.8503
Epoch 1210/2000
6680/6680 [==============================] - 8s - loss: 0.0130 - acc: 0.9990 - val_loss: 1.6342 - val_acc: 0.8599
Epoch 1211/2000
6680/6680 [==============================] - 8s - loss: 0.0110 - acc: 0.9993 - val_loss: 1.6171 - val_acc: 0.8515
Epoch 1212/2000
6680/6680 [==============================] - 8s - loss: 0.0097 - acc: 0.9994 - val_loss: 1.6745 - val_acc: 0.8479
Epoch 1213/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6082 - val_acc: 0.8515
Epoch 1214/2000
6680/6680 [==============================] - 8s - loss: 0.0141 - acc: 0.9991 - val_loss: 1.5777 - val_acc: 0.8539
Epoch 1215/2000
6680/6680 [==============================] - 8s - loss: 0.0110 - acc: 0.9993 - val_loss: 1.5722 - val_acc: 0.8527
Epoch 1216/2000
6680/6680 [==============================] - 8s - loss: 0.0123 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 7s - loss: 0.0127 - acc: 0.9991 - val_loss: 1.6792 - val_acc: 0.8515
Epoch 1275/2000
6680/6680 [==============================] - 7s - loss: 0.0116 - acc: 0.9993 - val_loss: 1.6356 - val_acc: 0.8539
Epoch 1276/2000
6680/6680 [==============================] - 8s - loss: 0.0115 - acc: 0.9991 - val_loss: 1.6277 - val_acc: 0.8503
Epoch 1277/2000
6680/6680 [==============================] - 8s - loss: 0.0078 - acc: 0.9993 - val_loss: 1.6266 - val_acc: 0.8503
Epoch 1278/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9991 - val_loss: 1.6064 - val_acc: 0.8551
Epoch 1279/2000
6680/6680 [==============================] - 8s - loss: 0.0150 - acc: 0.9990 - val_loss: 1.6621 - val_acc: 0.8563
Epoch 1280/2000
6680/6680 [==============================] - 7s - loss: 0.0145 - acc: 0.9990 - val_loss: 1.6606 - val_acc: 0.8515
Epoch 1281/2000
6680/6680 [==============================] - 8s - loss: 0.0144 - acc: 0.9988 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0113 - acc: 0.9991 - val_loss: 1.5837 - val_acc: 0.8515
Epoch 1340/2000
6680/6680 [==============================] - 8s - loss: 0.0130 - acc: 0.9990 - val_loss: 1.4382 - val_acc: 0.8635
Epoch 1341/2000
6680/6680 [==============================] - 8s - loss: 0.0130 - acc: 0.9990 - val_loss: 1.5401 - val_acc: 0.8659
Epoch 1342/2000
6680/6680 [==============================] - 7s - loss: 0.0110 - acc: 0.9991 - val_loss: 1.5549 - val_acc: 0.8575
Epoch 1343/2000
6680/6680 [==============================] - 8s - loss: 0.0095 - acc: 0.9993 - val_loss: 1.6555 - val_acc: 0.8527
Epoch 1344/2000
6680/6680 [==============================] - 7s - loss: 0.0116 - acc: 0.9987 - val_loss: 1.5682 - val_acc: 0.8611
Epoch 1345/2000
6680/6680 [==============================] - 7s - loss: 0.0141 - acc: 0.9988 - val_loss: 1.5847 - val_acc: 0.8599
Epoch 1346/2000
6680/6680 [==============================] - 7s - loss: 0.0123 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 7s - loss: 0.0119 - acc: 0.9990 - val_loss: 1.5613 - val_acc: 0.8575
Epoch 1373/2000
6680/6680 [==============================] - 7s - loss: 0.0105 - acc: 0.9993 - val_loss: 1.6459 - val_acc: 0.8503
Epoch 1374/2000
6680/6680 [==============================] - 7s - loss: 0.0135 - acc: 0.9990 - val_loss: 1.5953 - val_acc: 0.8575
Epoch 1375/2000
6680/6680 [==============================] - 7s - loss: 0.0136 - acc: 0.9990 - val_loss: 1.5942 - val_acc: 0.8503
Epoch 1376/2000
6680/6680 [==============================] - 7s - loss: 0.0127 - acc: 0.9991 - val_loss: 1.6329 - val_acc: 0.8515
Epoch 1377/2000
6680/6680 [==============================] - 8s - loss: 0.0118 - acc: 0.9993 - val_loss: 1.6356 - val_acc: 0.8515
Epoch 1378/2000
6680/6680 [==============================] - 7s - loss: 0.0110 - acc: 0.9991 - val_loss: 1.5139 - val_acc: 0.8551
Epoch 1379/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9990 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0102 - acc: 0.9991 - val_loss: 1.6259 - val_acc: 0.8539
Epoch 1438/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9988 - val_loss: 1.6534 - val_acc: 0.8383
Epoch 1439/2000
6680/6680 [==============================] - 7s - loss: 0.0109 - acc: 0.9990 - val_loss: 1.5948 - val_acc: 0.8515
Epoch 1440/2000
6680/6680 [==============================] - 7s - loss: 0.0141 - acc: 0.9985 - val_loss: 1.5515 - val_acc: 0.8563
Epoch 1441/2000
6680/6680 [==============================] - 7s - loss: 0.0114 - acc: 0.9990 - val_loss: 1.6101 - val_acc: 0.8539
Epoch 1442/2000
6680/6680 [==============================] - 8s - loss: 0.0090 - acc: 0.9993 - val_loss: 1.6608 - val_acc: 0.8527
Epoch 1443/2000
6680/6680 [==============================] - 7s - loss: 0.0136 - acc: 0.9988 - val_loss: 1.7231 - val_acc: 0.8407
Epoch 1444/2000
6680/6680 [==============================] - 8s - loss: 0.0094 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0147 - acc: 0.9990 - val_loss: 1.7078 - val_acc: 0.8491
Epoch 1471/2000
6680/6680 [==============================] - 8s - loss: 0.0127 - acc: 0.9991 - val_loss: 1.6566 - val_acc: 0.8587
Epoch 1472/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6256 - val_acc: 0.8479
Epoch 1473/2000
6680/6680 [==============================] - 8s - loss: 0.0103 - acc: 0.9993 - val_loss: 1.6461 - val_acc: 0.8563
Epoch 1474/2000
6680/6680 [==============================] - 8s - loss: 0.0122 - acc: 0.9991 - val_loss: 1.6356 - val_acc: 0.8515
Epoch 1475/2000
6680/6680 [==============================] - 8s - loss: 0.0126 - acc: 0.9991 - val_loss: 1.6512 - val_acc: 0.8455
Epoch 1476/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6530 - val_acc: 0.8527
Epoch 1477/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0137 - acc: 0.9990 - val_loss: 1.6578 - val_acc: 0.8563
Epoch 1536/2000
6680/6680 [==============================] - 7s - loss: 0.0149 - acc: 0.9988 - val_loss: 1.6702 - val_acc: 0.8503
Epoch 1537/2000
6680/6680 [==============================] - 7s - loss: 0.0107 - acc: 0.9993 - val_loss: 1.6734 - val_acc: 0.8539
Epoch 1538/2000
6680/6680 [==============================] - 7s - loss: 0.0100 - acc: 0.9993 - val_loss: 1.6898 - val_acc: 0.8635
Epoch 1539/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6682 - val_acc: 0.8575
Epoch 1540/2000
6680/6680 [==============================] - 8s - loss: 0.0142 - acc: 0.9990 - val_loss: 1.7078 - val_acc: 0.8551
Epoch 1541/2000
6680/6680 [==============================] - 7s - loss: 0.0094 - acc: 0.9993 - val_loss: 1.7941 - val_acc: 0.8503
Epoch 1542/2000
6680/6680 [==============================] - 7s - loss: 0.0130 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0087 - acc: 0.9993 - val_loss: 1.6557 - val_acc: 0.8395
Epoch 1601/2000
6680/6680 [==============================] - 7s - loss: 0.0118 - acc: 0.9990 - val_loss: 1.6563 - val_acc: 0.8467
Epoch 1602/2000
6680/6680 [==============================] - 7s - loss: 0.0131 - acc: 0.9990 - val_loss: 1.6572 - val_acc: 0.8551
Epoch 1603/2000
6680/6680 [==============================] - 8s - loss: 0.0129 - acc: 0.9991 - val_loss: 1.6782 - val_acc: 0.8527
Epoch 1604/2000
6680/6680 [==============================] - 7s - loss: 0.0128 - acc: 0.9990 - val_loss: 1.7095 - val_acc: 0.8395
Epoch 1605/2000
6680/6680 [==============================] - 7s - loss: 0.0140 - acc: 0.9987 - val_loss: 1.7069 - val_acc: 0.8551
Epoch 1606/2000
6680/6680 [==============================] - 7s - loss: 0.0128 - acc: 0.9991 - val_loss: 1.7620 - val_acc: 0.8407
Epoch 1607/2000
6680/6680 [==============================] - 8s - loss: 0.0116 - acc: 0.9993 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0125 - acc: 0.9988 - val_loss: 1.6277 - val_acc: 0.8563
Epoch 1666/2000
6680/6680 [==============================] - 8s - loss: 0.0143 - acc: 0.9990 - val_loss: 1.6545 - val_acc: 0.8515
Epoch 1667/2000
6680/6680 [==============================] - 7s - loss: 0.0116 - acc: 0.9990 - val_loss: 1.7372 - val_acc: 0.8587
Epoch 1668/2000
6680/6680 [==============================] - 8s - loss: 0.0145 - acc: 0.9990 - val_loss: 1.6840 - val_acc: 0.8575
Epoch 1669/2000
6680/6680 [==============================] - 8s - loss: 0.0124 - acc: 0.9991 - val_loss: 1.6796 - val_acc: 0.8575
Epoch 1670/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9991 - val_loss: 1.6614 - val_acc: 0.8575
Epoch 1671/2000
6680/6680 [==============================] - 7s - loss: 0.0102 - acc: 0.9993 - val_loss: 1.6584 - val_acc: 0.8623
Epoch 1672/2000
6680/6680 [==============================] - 8s - loss: 0.0095 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0131 - acc: 0.9990 - val_loss: 1.5640 - val_acc: 0.8611
Epoch 1699/2000
6680/6680 [==============================] - 8s - loss: 0.0099 - acc: 0.9993 - val_loss: 1.6701 - val_acc: 0.8515
Epoch 1700/2000
6680/6680 [==============================] - 8s - loss: 0.0145 - acc: 0.9990 - val_loss: 1.6586 - val_acc: 0.8551
Epoch 1701/2000
6680/6680 [==============================] - 8s - loss: 0.0118 - acc: 0.9993 - val_loss: 1.6974 - val_acc: 0.8467
Epoch 1702/2000
6680/6680 [==============================] - 8s - loss: 0.0136 - acc: 0.9990 - val_loss: 1.6578 - val_acc: 0.8455
Epoch 1703/2000
6680/6680 [==============================] - 8s - loss: 0.0115 - acc: 0.9993 - val_loss: 1.6316 - val_acc: 0.8587
Epoch 1704/2000
6680/6680 [==============================] - 8s - loss: 0.0137 - acc: 0.9991 - val_loss: 1.7021 - val_acc: 0.8539
Epoch 1705/2000
6680/6680 [==============================] - 8s - loss: 0.0132 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0127 - acc: 0.9991 - val_loss: 1.6441 - val_acc: 0.8479
Epoch 1732/2000
6680/6680 [==============================] - 8s - loss: 0.0090 - acc: 0.9993 - val_loss: 1.6821 - val_acc: 0.8539
Epoch 1733/2000
6680/6680 [==============================] - 8s - loss: 0.0109 - acc: 0.9993 - val_loss: 1.6426 - val_acc: 0.8587
Epoch 1734/2000
6680/6680 [==============================] - 8s - loss: 0.0126 - acc: 0.9991 - val_loss: 1.6617 - val_acc: 0.8515
Epoch 1735/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9991 - val_loss: 1.6802 - val_acc: 0.8467
Epoch 1736/2000
6680/6680 [==============================] - 8s - loss: 0.0129 - acc: 0.9988 - val_loss: 1.6930 - val_acc: 0.8455
Epoch 1737/2000
6680/6680 [==============================] - 7s - loss: 0.0107 - acc: 0.9991 - val_loss: 1.6852 - val_acc: 0.8527
Epoch 1738/2000
6680/6680 [==============================] - 8s - loss: 0.0136 - acc: 0.9991 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0115 - acc: 0.9993 - val_loss: 1.7108 - val_acc: 0.8407
Epoch 1765/2000
6680/6680 [==============================] - 8s - loss: 0.0123 - acc: 0.9991 - val_loss: 1.7017 - val_acc: 0.8515
Epoch 1766/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9991 - val_loss: 1.6771 - val_acc: 0.8527
Epoch 1767/2000
6680/6680 [==============================] - 8s - loss: 0.0128 - acc: 0.9991 - val_loss: 1.6903 - val_acc: 0.8443
Epoch 1768/2000
6680/6680 [==============================] - 8s - loss: 0.0116 - acc: 0.9993 - val_loss: 1.6224 - val_acc: 0.8551
Epoch 1769/2000
6680/6680 [==============================] - 8s - loss: 0.0135 - acc: 0.9990 - val_loss: 1.6920 - val_acc: 0.8443
Epoch 1770/2000
6680/6680 [==============================] - 8s - loss: 0.0104 - acc: 0.9993 - val_loss: 1.7178 - val_acc: 0.8431
Epoch 1771/2000
6680/6680 [==============================] - 8s - loss: 0.0110 - acc: 0.9993 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0138 - acc: 0.9990 - val_loss: 1.6460 - val_acc: 0.8479
Epoch 1830/2000
6680/6680 [==============================] - 8s - loss: 0.0114 - acc: 0.9991 - val_loss: 1.6509 - val_acc: 0.8491
Epoch 1831/2000
6680/6680 [==============================] - 8s - loss: 0.0124 - acc: 0.9988 - val_loss: 1.6275 - val_acc: 0.8575
Epoch 1832/2000
6680/6680 [==============================] - 8s - loss: 0.0113 - acc: 0.9993 - val_loss: 1.7183 - val_acc: 0.8491
Epoch 1833/2000
6680/6680 [==============================] - 8s - loss: 0.0117 - acc: 0.9991 - val_loss: 1.6286 - val_acc: 0.8551
Epoch 1834/2000
6680/6680 [==============================] - 8s - loss: 0.0125 - acc: 0.9991 - val_loss: 1.6471 - val_acc: 0.8503
Epoch 1835/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.6184 - val_acc: 0.8563
Epoch 1836/2000
6680/6680 [==============================] - 8s - loss: 0.0137 - acc: 0.9990 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0141 - acc: 0.9988 - val_loss: 1.6115 - val_acc: 0.8575
Epoch 1895/2000
6680/6680 [==============================] - 8s - loss: 0.0123 - acc: 0.9991 - val_loss: 1.5682 - val_acc: 0.8611
Epoch 1896/2000
6680/6680 [==============================] - 8s - loss: 0.0118 - acc: 0.9993 - val_loss: 1.6372 - val_acc: 0.8539
Epoch 1897/2000
6680/6680 [==============================] - 8s - loss: 0.0112 - acc: 0.9991 - val_loss: 1.6166 - val_acc: 0.8635
Epoch 1898/2000
6680/6680 [==============================] - 8s - loss: 0.0131 - acc: 0.9991 - val_loss: 1.6875 - val_acc: 0.8563
Epoch 1899/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9993 - val_loss: 1.6204 - val_acc: 0.8671
Epoch 1900/2000
6680/6680 [==============================] - 8s - loss: 0.0115 - acc: 0.9993 - val_loss: 1.6285 - val_acc: 0.8575
Epoch 1901/2000
6680/6680 [==============================] - 8s - loss: 0.0102 - acc: 0.9993 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0111 - acc: 0.9993 - val_loss: 1.6193 - val_acc: 0.8419
Epoch 1960/2000
6680/6680 [==============================] - 8s - loss: 0.0143 - acc: 0.9990 - val_loss: 1.6958 - val_acc: 0.8539
Epoch 1961/2000
6680/6680 [==============================] - 8s - loss: 0.0103 - acc: 0.9993 - val_loss: 1.6589 - val_acc: 0.8527
Epoch 1962/2000
6680/6680 [==============================] - 8s - loss: 0.0113 - acc: 0.9987 - val_loss: 1.7299 - val_acc: 0.8395
Epoch 1963/2000
6680/6680 [==============================] - 8s - loss: 0.0117 - acc: 0.9991 - val_loss: 1.6462 - val_acc: 0.8479
Epoch 1964/2000
6680/6680 [==============================] - 8s - loss: 0.0110 - acc: 0.9990 - val_loss: 1.6166 - val_acc: 0.8539
Epoch 1965/2000
6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.7364 - val_acc: 0.8527
Epoch 1966/2000
6680/6680 [==============================] - 8s - loss: 0.0104 - acc: 0.9990 - val_loss: 1

6680/6680 [==============================] - 8s - loss: 0.0121 - acc: 0.9993 - val_loss: 1.5530 - val_acc: 0.8563
Epoch 1993/2000
6680/6680 [==============================] - 8s - loss: 0.0148 - acc: 0.9990 - val_loss: 1.6246 - val_acc: 0.8587
Epoch 1994/2000
6680/6680 [==============================] - 8s - loss: 0.0120 - acc: 0.9990 - val_loss: 1.7213 - val_acc: 0.8491
Epoch 1995/2000
6680/6680 [==============================] - 8s - loss: 0.0099 - acc: 0.9987 - val_loss: 1.6690 - val_acc: 0.8527
Epoch 1996/2000
6680/6680 [==============================] - 8s - loss: 0.0112 - acc: 0.9987 - val_loss: 1.7038 - val_acc: 0.8563
Epoch 1997/2000
6680/6680 [==============================] - 8s - loss: 0.0127 - acc: 0.9991 - val_loss: 1.6514 - val_acc: 0.8575
Epoch 1998/2000
6680/6680 [==============================] - 8s - loss: 0.0124 - acc: 0.9991 - val_loss: 1.6740 - val_acc: 0.8527
Epoch 1999/2000
6680/6680 [==============================] - 8s - loss: 0.0136 - acc: 0.9985 - val_loss: 1

### (IMPLEMENTATION) Load the Model with the Best Validation Loss

In [34]:
### TODO: Load the model weights with the best validation loss.
Inc_model.load_weights('saved_models/weights.best.Inc_model.hdf5')

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Ensure that your test accuracy is greater than 60%.

In [36]:
### TODO: Calculate classification accuracy on the test dataset.
Inc_predictions = [np.argmax(Inc_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Inception]

# report test accuracy
test_accuracy = 100*np.sum(np.array(Inc_predictions)==np.argmax(test_targets, axis=1))/len(Inc_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.8182%


### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan_hound`, etc) that is predicted by your model.  

Similar to the analogous function in Step 5, your function should have three steps:
1. Extract the bottleneck features corresponding to the chosen CNN model.
2. Supply the bottleneck features as input to the model to return the predicted vector.  Note that the argmax of this prediction vector gives the index of the predicted dog breed.
3. Use the `dog_names` array defined in Step 0 of this notebook to return the corresponding breed.

The functions to extract the bottleneck features can be found in `extract_bottleneck_features.py`, and they have been imported in an earlier code cell.  To obtain the bottleneck features corresponding to your chosen CNN architecture, you need to use the function

    extract_{network}
    
where `{network}`, in the above filename, should be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.

In [40]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

def Inc_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_InceptionV3(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = Inc_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]
    

---
<a id='step6'></a>
## Step 6: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `dog_detector` functions developed above.  You are __required__ to use your CNN from Step 5 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [41]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

class NeitherDogNorHumanError(Exception):
    """Subclassing Error for clarity"""
    pass

def predict_breed_or_resemblance(img_path):
    creature = None
    message = None
    breed = None
    if dog_detector(img_path):
        creature = "dog"
        message = "your predicted breed is"
    
    if face_detector(img_path):
        creature = "human"
        message = "you look like a"
    
    if not creature:
        raise NeitherDogNorHumanError("Tis neither a dog nor a human!!!")
    
    breed = Inc_predict_breed(img_path)
    return "Hello, {}! {} ... {}".format(creature, message, breed)

---
<a id='step7'></a>
## Step 7: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that __you__ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ 

In [42]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

"""
Answer 6:

The output is better than I expected :) On the 6 images I tried, the accuracy is 100%.
On a funny note, I've been living in Ireland for 12 years and my CNN found me to be similar to an Irish water spaniel
quite consistently. (And found Mark Zuckerberg to be an Australian terrier).


These are some possible improvements to the algorithm:
1. store the probabilities of a prediction (just before they get 'softmax-ed') and determine dog-or-human based on
the highest probability of the classification rather than short-circuiting at dog/human detection
2. another possible improvement is to build several models and perform prediction on an ensemble model
3. another possible improvement - related to the mistaken classification of an animal for a dog (or a human) is
to set a minimum threshold of certainty. Even in case of a picture being classified as a dog, if the max probability
of the picture to be a particular breed is below an arbitrary threshold (say 50%) then reject the classification and 
raise the NeitherDogNorHumanError exception.
"""

me = "./doomsword_deathmaster.jpg"
## testing...
print(predict_breed_or_resemblance("./dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg"))
print(predict_breed_or_resemblance(me))
print(predict_breed_or_resemblance("./Mark-Zuckerberg.jpeg"))
print(predict_breed_or_resemblance("./images/Brittany_02625.jpg"))
print(predict_breed_or_resemblance("./images/Curly-coated_retriever_03896.jpg"))
print(predict_breed_or_resemblance("./images/Labrador_retriever_06449.jpg"))

Hello, dog! your predicted breed is ... Affenpinscher
Hello, human! you look like a ... Irish_water_spaniel
Hello, human! you look like a ... Australian_terrier
Hello, dog! your predicted breed is ... Brittany
Hello, dog! your predicted breed is ... Curly-coated_retriever
Hello, dog! your predicted breed is ... Labrador_retriever
